In [1]:
import bz2
import os
import re

import mwparserfromhell
import numpy as np
import pandas as pd
import requests

import findspark
findspark.init('/usr/lib/spark2')

from pyspark.sql import SparkSession

In [1]:
!which python

/usr/lib/anaconda-wmf/bin/python


In [2]:
regular = True

if regular:
    spark = (
        SparkSession.builder
        .appName('Pyspark notebook (isaacj -- article data)')
        .master('yarn')
        .config(
            'spark.driver.extraJavaOptions',
            ' '.join('-D{}={}'.format(k, v) for k, v in {
                'http.proxyHost': 'webproxy.eqiad.wmnet',
                'http.proxyPort': '8080',
                'https.proxyHost': 'webproxy.eqiad.wmnet',
                'https.proxyPort': '8080',
            }.items()))
    #    .config('spark.jars.packages', 'graphframes:graphframes:0.6.0-spark2.3-s_2.11')
        .config("spark.driver.memory", "2g")
        .config('spark.dynamicAllocation.maxExecutors', 64)
        .config("spark.executor.memory", "8g")
        .config("spark.executor.cores", 4)
        .config("spark.sql.shuffle.partitions", 256)
        .getOrCreate()
    )
else:
    spark = (
        SparkSession.builder
        .appName('Pyspark notebook (isaacj -- article data)')
        .master('yarn')
        .config(
            'spark.driver.extraJavaOptions',
            ' '.join('-D{}={}'.format(k, v) for k, v in {
                'http.proxyHost': 'webproxy.eqiad.wmnet',
                'http.proxyPort': '8080',
                'https.proxyHost': 'webproxy.eqiad.wmnet',
                'https.proxyPort': '8080',
            }.items()))
    #    .config('spark.jars.packages', 'graphframes:graphframes:0.6.0-spark2.3-s_2.11')
        .config("spark.driver.memory", "4g")
        .config('spark.dynamicAllocation.maxExecutors', 128)
        .config("spark.executor.memory", "8g")
        .config("spark.executor.cores", 4)
        .config("spark.sql.shuffle.partitions", 512)
        .getOrCreate()
    )
spark

In [21]:
snapshot = '2020-11'  # data will be current to this date -- e.g., 2020-07 means data is up to 30 June 2020 (at least)
wd_snapshot = '2020-11-02'
wiki = 'enwiki'  # wikidb you want to run pagerank for
year = snapshot.split('-')[0]
month = str(int(snapshot.split('-')[1]))

## Attributes
All of these calls have the following in common:
* Use pageID as the unique key
* Namespace 0 only
* Redirects should be resolved if that's relevant

In [4]:
def getLinksMWP(wikitext):
    """Extract list of links from wikitext for an article via mwparserfromhell."""
    try:
        wt = mwparserfromhell.parse(wikitext)
        return [str(l.title).partition('#')[0].replace(' ', '_').strip() for l in wt.filter_wikilinks()]
    except Exception:
        return None
    
spark.udf.register('getLinks', getLinksMWP, 'Array<String>')

<function __main__.getLinksMWP(wikitext)>

In [5]:
sfn_templates = ["Shortened footnote template", "sfn", "Sfnp", "Sfnm", "Sfnmp"]

def getRefsMWP(wikitext):
    """Extract list of links from wikitext for an article via mwparserfromhell."""
    try:
        wt = mwparserfromhell.parse(wikitext)
        num_ref_tags = len([t.tag for t in wt.filter_tags() if t.tag == 'ref'])
        num_sftn_templates = len([t.name for t in mwparserfromhell.parse(wikitext).filter_templates() if t.name in sfn_templates])
        return num_ref_tags + num_sftn_templates
    except Exception:
        return None
    
spark.udf.register('getNumRefs', getRefsMWP, 'Int')

<function __main__.getRefsMWP(wikitext)>

In [6]:
def getNamespacePrefixes(lang):
    session = requests.Session()
    base_url = "https://{0}.wikipedia.org/w/api.php".format(lang)
    params = {
        "action": "query",
        "meta": "siteinfo",
        "siprop": "namespacealiases|namespaces",
        "format": "json",
        "formatversion": "2"
    }
    result = session.get(url=base_url, params=params)
    result = result.json()
    prefix_to_ns = {}
    if 'namespacealiases' in result.get('query', {}):
        for alias in result['query']['namespacealiases']:
            prefix_to_ns[alias['alias']] = alias['id']
    if 'namespaces' in result.get('query', {}):
        for ns in result['query']['namespaces'].values():
            if 'name' in ns:
                prefix_to_ns[ns['name'].replace(' ', '_')] = ns['id']
            if 'canonical' in ns:
                prefix_to_ns[ns['canonical'].replace(' ', '_')] = ns['id']
    return prefix_to_ns

namespace_prefixes = getNamespacePrefixes(wiki.replace('wiki', ''))

def filterLinksByNs(links, keep_ns):
    for i in range(len(links)-1, -1, -1):
        link_ns = 0
        if ':' in links[i]:
            prefix = links[i].split(':')[0].replace(' ', '_')
            if prefix in namespace_prefixes:
                link_ns = namespace_prefixes[prefix]
        if link_ns not in keep_ns:
            links.pop(i)
    return links

spark.udf.register('filterLinksByNs', filterLinksByNs, 'Array<String>')

<function __main__.filterLinksByNs(links, keep_ns)>

In [7]:
def getHeadingsMWP(wikitext, level=None):
    """Extract list of headings from wikitext for an article."""
    try:
        wt = mwparserfromhell.parse(wikitext)
        if level is None:
            return [str(l.title).strip() for l in wt.filter_headings()]
        else:
            return [str(l.title).strip() for l in wt.filter_headings() if l.level == level]
    except Exception:
        return None
    
spark.udf.register('getHeadings', getHeadingsMWP, 'Array<String>')

<function __main__.getHeadingsMWP(wikitext, level=None)>

In [8]:
def getTemplatesMWP(wikitext):
    """Extract list of templates from wikitext for an article via mwparserfromhell."""
    try:
        wt = mwparserfromhell.parse(wikitext)
        return [str(t.name).replace(' ', '_').strip() for t in wt.filter_templates()]
    except Exception:
        return None
    
spark.udf.register('getTemplates', getTemplatesMWP, 'Array<String>')

<function __main__.getTemplatesMWP(wikitext)>

In [9]:
def max_quality(quality_scores):
    ranking = {'N/A':-2, 'Unrated':-1, 'Stub':0, 'Start':1, 'C':2, 'B':3, 'GA':4, 'FA':5}
    return sorted(quality_scores, key=lambda x: ranking.get(x, -3), reverse=True)[0]

spark.udf.register('max_quality', max_quality, 'String')

def max_importance(importance_scores):
    ranking = {'N/A':-2, 'Unrated':-1, 'Low':0, 'Mid':1, 'High':2, 'Top':3}
    return sorted(importance_scores, key=lambda x: ranking.get(x, -3), reverse=True)[0]

spark.udf.register('max_importance', max_importance, 'String')

<function __main__.max_importance(importance_scores)>

In [10]:
"""
Explanation of CTEs:
* title_to_id: mapping of page title to page ID, which is a more stable identifier
* redirects: resolve redirects so the network is much denser (~6M nodes instead of ~11M nodes)
* pagelinks: extract links from wikitext and explode so each row has one link
* pagelinks_reformatted: map link page titles to link page IDs
* final: resolve redirects and rename columns to match pagerank library expectations
* https://github.com/wikimedia/articlequality/blob/master/articlequality/feature_lists/enwiki.py
"""

print_for_hive = False
do_execute = True

query = """
WITH pageid_to_qid AS (
    SELECT
      item_id,
      page_id,
      page_title
    FROM wmf.wikidata_item_page_link
    WHERE
      snapshot = '{0}'
      AND wiki_db = '{1}'
      AND page_namespace = 0
),
jazz_wikiproject AS (
    SELECT
      pa.page_id AS page_id,
      MAX(wd.item_id) AS qid,
      MAX(wd.page_title) AS page_title,
      max_importance(COLLECT_LIST(importance)) AS importance
    FROM isaacj.page_assessments_enwiki pa
    INNER JOIN pageid_to_qid wd
      ON (pa.page_id = wd.page_id)
    WHERE
      wp_template = 'Jazz'
    GROUP BY
      pa.page_id
),
title_to_id AS (
    SELECT
      page_id,
      page_title
    FROM wmf_raw.mediawiki_page
    WHERE
      page_namespace = 0
      AND snapshot = '{2}'
      AND wiki_db = '{1}'
),
redirects AS (
    SELECT
      mr.rd_from AS rd_from,
      tti.page_id AS rd_to
    FROM wmf_raw.mediawiki_redirect mr
    INNER JOIN title_to_id tti
      ON (mr.rd_title = tti.page_title)
    WHERE
      mr.snapshot = '{2}'
      AND mr.rd_namespace = 0
      AND mr.wiki_db = '{1}'
),
pagelinks_reformatted AS (
    SELECT
      pl.pl_from AS pl_from,
      tti.page_id AS pl_to
    FROM wmf_raw.mediawiki_pagelinks pl
    INNER JOIN title_to_id tti
      ON (pl.pl_title = tti.page_title)
    LEFT ANTI JOIN redirects r
      ON (pl.pl_from = r.rd_from)
    WHERE
      pl.snapshot = '{2}'
      AND pl.pl_from_namespace = 0
      AND pl.pl_namespace = 0
      AND pl.wiki_db = '{1}'
),
links AS (
    SELECT
      pl.pl_from AS src,
      COALESCE(r.rd_to, pl.pl_to) AS dst
    FROM pagelinks_reformatted pl
    LEFT JOIN redirects r
      ON (pl.pl_to = r.rd_from)
),
inlinks AS (
    SELECT
      l.dst AS page_id,
      SIZE(COLLECT_SET(l.src)) AS num_inlinks
    FROM links l
    INNER JOIN jazz_wikiproject j
      ON (l.dst = j.page_id)
    GROUP BY
      l.dst
),
outlinks AS (
    SELECT
      l.src AS page_id,
      SIZE(COLLECT_SET(l.dst)) AS num_outlinks
    FROM links l
    INNER JOIN jazz_wikiproject j
      ON (l.src = j.page_id)
    GROUP BY
      l.src
),
pageviews AS (
    SELECT
      pv.page_id AS page_id,
      SUM(pv.view_count) AS monthly_pageviews
    FROM wmf.pageview_hourly pv
    INNER JOIN jazz_wikiproject j
      ON (pv.page_id = j.page_id)
    WHERE
      AND pv.project = '{3}.wikipedia'
      AND year = {4}
      AND month = {5}
      AND agent_type = 'user'
)
region_data AS (
    SELECT
      r.qid AS qid,
      CONCAT_WS('|', COLLECT_LIST(r.region)) AS regions
    FROM isaacj.qid_to_country r
    INNER JOIN jazz_wikiproject j
      ON (r.qid = j.qid)
    GROUP BY
      r.qid
),
gender_data AS (
    SELECT
      g.item_id AS qid,
      g.value AS gender
    FROM isaacj.gender_wikidata g
    INNER JOIN jazz_wikiproject j
      ON (g.item_id = j.qid)
    WHERE
      g.property = 'P21'
),
wikitext AS (
    SELECT
      wt.page_id AS page_id,
      wt.revision_text AS wikitext
    FROM wmf.mediawiki_wikitext_current wt
    INNER JOIN jazz_wikiproject j
      ON (wt.page_id = j.page_id)
    WHERE
      wt.snapshot = '{2}'
      AND wt.wiki_db = '{1}'
)
SELECT
  j.page_id AS page_id,
  j.qid AS qid,
  CONCAT('https://en.wikipedia.org/wiki/', j.page_title) AS url,
  j.importance AS importance,
  g.gender AS gender,
  r.regions AS countries,
  LENGTH(wt.wikitext) AS page_length,
  SIZE(getHeadings(wt.wikitext)) AS num_sections,
  SIZE(getTemplates(wt.wikitext)) AS num_templates,
  getNumRefs(wt.wikitext) AS num_refs,
  i.num_inlinks AS num_inlinks,
  o.num_outlinks AS num_outlinks,
  pv.monthly_pageviews AS monthly_pageviews
FROM jazz_wikiproject j
LEFT JOIN region_data r
  ON (j.qid = r.qid)
LEFT JOIN gender_data g
  ON (j.qid = g.qid)
LEFT JOIN wikitext wt
  ON (j.page_id = wt.page_id)
LEFT JOIN inlinks i
  ON (j.page_id = i.page_id)
LEFT JOIN outlinks o
  ON (j.page_id = o.page_id)
LEFT JOIN pageviews pv
  ON (j.page_id = pv.page_id)

""".format(wd_snapshot, wiki, snapshot, wiki.replace('wiki', ''), year, month)

if print_for_hive:
    print(re.sub(' +', ' ', re.sub('\n', ' ', query)).strip())
else:
    print(query)

if do_execute:
    result = spark.sql(query)


WITH pageid_to_qid AS (
    SELECT
      item_id,
      page_id,
      page_title
    FROM wmf.wikidata_item_page_link
    WHERE
      snapshot = '2020-11-02'
      AND wiki_db = 'enwiki'
      AND page_namespace = 0
),
jazz_wikiproject AS (
    SELECT
      pa.page_id AS page_id,
      MAX(wd.item_id) AS qid,
      MAX(wd.page_title) AS page_title,
      max_importance(COLLECT_LIST(importance)) AS importance
    FROM isaacj.page_assessments_enwiki pa
    INNER JOIN pageid_to_qid wd
      ON (pa.page_id = wd.page_id)
    WHERE
      wp_template = 'Jazz'
    GROUP BY
      pa.page_id
),
title_to_id AS (
    SELECT
      page_id,
      page_title
    FROM wmf_raw.mediawiki_page
    WHERE
      page_namespace = 0
      AND snapshot = '2020-11'
      AND wiki_db = 'enwiki'
),
redirects AS (
    SELECT
      mr.rd_from AS rd_from,
      tti.page_id AS rd_to
    FROM wmf_raw.mediawiki_redirect mr
    INNER JOIN title_to_id tti
      ON (mr.rd_title = tti.page_title)
    WHERE
      mr.snaps

In [12]:
# write pagerank results to TSV
!hdfs dfs -rm -R jazz-data
result.write.csv(path="/user/isaacj/jazz-data", compression="bzip2", header=True, sep="\t")

20/12/22 14:29:12 INFO fs.TrashPolicyDefault: Moved: 'hdfs://analytics-hadoop/user/isaacj/jazz-data' to trash at: hdfs://analytics-hadoop/user/isaacj/.Trash/Current/user/isaacj/jazz-data1608647352103


In [17]:
file_parts_dir = './file_parts/'
!rm -R {file_parts_dir}
!mkdir {file_parts_dir}
!hdfs dfs -copyToLocal jazz-data/part* {file_parts_dir}

In [18]:
fns = [fn for fn in os.listdir(file_parts_dir) if fn.endswith('.csv.bz2')]
jazz_data_combined = 'jazz_article_data.tsv'
print_every = 1
processed = 0
output_header = ['page_id', 'qid', 'url',
                 'importance',
                 'gender', 'countries',
                 'page_length', 'num_sections', 'num_templates', 'num_refs', 'num_inlinks', 'num_outlinks',
                 'monthly_pageviews']
with open(jazz_data_combined, 'w') as fout:
    fout.write('\t'.join(output_header) + '\n')
    for i, fn in enumerate(fns, start=1):
        with bz2.open(os.path.join(file_parts_dir, fn), 'rt') as fin:
            header = next(fin).strip().split('\t')
            assert header == output_header
            for line_no, line_str in enumerate(fin, start=1):
                line = line_str.strip().split('\t')
                assert len(line) == len(output_header)
                processed += 1
                fout.write(line_str)
        if i % print_every == 0:
            print("{0} / {1} files processed.".format(i, len(fns)))
            print_every = print_every * 2
print("{0} pages processed.".format(processed))

1 / 256 files processed.
2 / 256 files processed.
4 / 256 files processed.
8 / 256 files processed.
16 / 256 files processed.
32 / 256 files processed.
64 / 256 files processed.
128 / 256 files processed.
256 / 256 files processed.
26877 pages processed.


In [20]:
!grep -P '/John_Coltrane\t' jazz_article_data.tsv

53536	Q7346	https://en.wikipedia.org/wiki/John_Coltrane	Top	Q6581097	United States of America	72751	23	129	136	2880	349


## Regex-based (i.e. no mwparserfromhell dependency) versions
NOTE: these functions are generally good replacements and in theory are faster / easier to deploy to workers because of their simplicity but do not catch as many edge-cases as mwparserfromhell does so if you have the luxury of using mwparserfromhell, go with it.

In [ ]:
def getTemplatesRegex(wikitext):
    """Extract list of templates from wikitext for an article via simple regex.
    Known Issues:
    * Doesn't handle nested templates (just gets the first)
    -- e.g., '{{cite web|url=http://www.allmusic.com/|ref={{harvid|AllMusic}}}}' would be just web
    """
    try:
        return [m.split('|')[0].replace(' ', '_').strip() for m in re.findall('(?<=\{\{)(.*?)(?=\}\})', wikitext, flags=re.DOTALL)]
    except Exception:
        return None
    
spark.udf.register('getTemplates', getTemplatesRegex, 'Array<String>')

def getHeadingsRegex(wikitext, level=None):
    """Extract list of headings from wikitext for an article."""
    try:
        wt = mwparserfromhell.parse(wikitext)
        if level is None:
            return [l[1].strip() for l in re.findall('(={2,})(.*?)(={2,})', wikitext)]
        else:
            return [l[1].strip() for l in re.findall('(={2,})(.*?)(={2,})', wikitext) if len(l[0]) == level]
    except Exception:
        return None
    
spark.udf.register('getHeadings', getHeadingsRegex, 'Array<String>')

def getLinksRegex(wikitext):
    """Extract list of links from wikitext for an article via simple regex.
    Known Issues:
    * Doesn't handle nested wikilinks (just gets the first) -- e.g., [[File:filename|caption=[[second_wikilink]]]] would be just File:filename
    """
    try:
        return [m.split('|')[0].partition('#')[0].replace(' ', '_').strip() for m in re.findall('(?<=\[\[)(.*?)(?=\]\])', wikitext, flags=re.DOTALL)]
    except Exception:
        return None
    
spark.udf.register('getLinks', getLinksRegex, 'Array<String>')

In [29]:
# Anarchism article is great for debugging because it's super complex
wikitext = """
{{short description|Political philosophy and movement}}
{{redirect2|Anarchist|Anarchists|other uses|Anarchists (disambiguation)}}
{{pp-move-indef}}
{{good article}}
{{use dmy dates|date=March 2020}}
{{use British English|date=January 2014}}
{{anarchism sidebar}}
{{basic forms of government}}
'''Anarchism''' is a [[political philosophy]] and [[Political movement|movement]] that is sceptical of [[authority]] and rejects all involuntary, coercive forms of [[hierarchy]]. Anarchism calls for the abolition of the [[State (polity)|state]], which it holds to be undesirable, unnecessary, and harmful. It is usually described alongside [[libertarian Marxism]] as the [[libertarian]] wing ([[libertarian socialism]]) of the [[socialist movement]] and as having a historical association with [[anti-capitalism]] and [[socialism]].

The [[history of anarchism]] goes back to [[prehistory]], when humans arguably lived in anarchistic societies long before the establishment of formal states, [[realm]]s or [[empire]]s. With the rise of organised hierarchical bodies, [[scepticism]] toward authority also rose, but it was not until the 19th century that a self-conscious political movement emerged. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement flourished in most parts of the world and had a significant role in workers' struggles for [[emancipation]]. Various [[anarchist schools of thought]] formed during this period. Anarchists have taken part in several revolutions, most notably in the [[Spanish Civil War]], whose end marked the end of the [[classical era of anarchism]]. In the last decades of the 20th century and into the 21st century, the anarchist movement has been resurgent once more.

Anarchism employs a [[diversity of tactics]] in order to meet its ideal ends which can be broadly separated into revolutionary and evolutionary tactics. There is significant overlap between the two which are merely descriptive. Revolutionary tactics aim to bring down authority and state, having taken a violent turn in the past. Evolutionary tactics aim to prefigure what an anarchist society would be like. Anarchist thought, criticism and [[Praxis (process)|praxis]] have played a part in diverse areas of human society. Criticism of anarchism mainly focuses on claims of it being internally inconsistent, violent and utopian.
{{toc limit|3}}

== Etymology, terminology and definition ==
{{main|Definition of anarchism and libertarianism}}
{{see also|Glossary of anarchism}}
[[File:WilhelmWeitling.jpg|thumb|[[Wilhelm Weitling]], an example of a writer who added to anarchist theory without using the exact term]]
The etymological origin of ''anarchism'' is from the Ancient Greek ''anarkhia'', meaning "without a ruler", composed of the prefix ''an-'' (i.e. "without") and the word ''arkhos'' (i.e. "leader" or "ruler"). The suffix ''[[-ism]]'' denotes the ideological current that favours [[anarchy]].{{sfnm|1a1=Bates|1y=2017|1p=128|2a1=Long|2y=2013|2p=217}} ''Anarchism'' appears in English from 1642 as ''anarchisme'' and ''anarchy'' from 1539; early English usages empahised a sense of disorder.{{sfnm|1a1=Merriam-Webster|1y=2019|1loc="Anarchism"|2a1=''Oxford English Dictionary''|2y=2005|2loc="Anarchism"|3a1=Sylvan|3y=2007|3p=260}} Various factions within the [[French Revolution]] labelled their opponents as ''anarchists'', although few such accused shared many views with later anarchists. Many revolutionaries of the 19th century such as [[William Godwin]] (1756–1836) and [[Wilhelm Weitling]] (1808–1871) would contribute to the anarchist doctrines of the next generation, but they did not use ''anarchist'' or ''anarchism'' in describing themselves or their beliefs.{{sfn|Joll|1964|pp=27–37}}

The first political philosopher to call himself an ''anarchist'' ({{lang-fr|anarchiste}}) was [[Pierre-Joseph Proudhon]] (1809–1865), marking the formal birth of anarchism in the mid-19th century. Since the 1890s and beginning in France,{{sfn|Nettlau|1996|p=162}} ''[[libertarianism]]'' has often been used as a synonym for anarchism{{sfn|Guérin|1970|loc="The Basic Ideas of Anarchism"}} and its use as a synonym is still common outside the United States.{{sfnm|1a1=Ward|1y=2004|1p=62|2a1=Goodway|2y=2006|2p=4|3a1=Skirda|3y=2002|3p=183|4a1=Fernández|4y=2009|4p=9}} On the other hand, some use ''libertarianism'' to refer to [[individualistic]] [[free-market]] philosophy only, referring to [[free-market anarchism]] as ''libertarian anarchism''.{{sfn|Morris|2002|p=61}}

While the term ''libertarian'' has been largely synonymous with anarchism,{{sfnm|1a1=Marshall|1y=1992|1p=641|2a1=Cohn|2y=2009|2p=6}} its meaning has more recently diluted with wider adoption from ideologically disparate groups,{{sfn|Marshall|1992|p=641}} including both the [[New Left]] and [[libertarian Marxists]] (who do not associate themselves with [[authoritarian socialists]] or a [[vanguard party]]) as well as extreme [[Liberalism|liberals]] (primarily concerned with [[civil liberties]]).{{sfn|Marshall|1992|p=641}} Additionally, some anarchists use ''[[libertarian socialist]]''{{sfnm|1a1=Marshall|1y=1992|1p=641|2a1=Chomsky|2y=2005|2p=123|3a1=Cohn|3y=2009|3p=6|4a1=Adams|4a2=Levy|4y=2018|4p=104}} to avoid anarchism's negative connotations and emphasise its connections with [[socialism]].{{sfn|Marshall|1992|p=641}} Matthew S. Adams and [[Carl Levy (political scientist)|Carl Levy]] write that ''anarchism'' is used to "describe the [[anti-authoritarian]] wing of the [[socialist movement]]".{{sfn|Adams|Levy|2018|p=104}} [[Noam Chomsky]] describes anarchism, alongside libertarian Marxism, as "the libertarian wing of socialism".{{sfn|Chomsky|2005|p=123}} [[Daniel Guérin]] wrote:
<blockquote>[A]narchism is really a synonym for socialism. The anarchist is primarily a socialist whose aim is to abolish the exploitation of man by man. Anarchism is only one of the streams of socialist thought, that stream whose main components are concern for liberty and haste to abolish the State.{{sfn|Guérin|1970|p=12}}</blockquote>

While [[opposition to the state]] is central to anarchist thought, defining anarchism is not an easy task as there is a lot of discussion among scholars and anarchists on the matter and various currents perceive anarchism slightly differently.{{sfn|Long|2013|p=217}} Hence, it might be true to say that anarchism is a cluster of political philosophies opposing [[authority]] and [[hierarchical organisation]] (including [[Anarchism and capitalism|capitalism]], [[Anarchism and nationalism|nationalism]], the [[State (polity)|state]]  and all associated [[institution]]s) in the conduct of all [[human relations]] in favour of a society based on [[decentralisation]], [[freedom]] and [[voluntary association]]. However, this definition has the same shortcomings as the definition based on anti-authoritarianism (which is an ''[[a posteriori]]'' conclusion), anti-statism (anarchism is much more than that){{sfnm|1a1=McLaughlin|1y=2007|1p=166|2a1=Jun|2y=2009|2p=507|3a1=Franks|3y=2013|3pp=386–388}} and etymology (which is simply a negation of a ruler).{{sfnm|1a1=McLaughlin|1y=2007|1pp=25–29|2a1=Long|2y=2013|2pp=217}} Nonetheless, major elements of the definition of anarchism include the will for a non-coercive society, the rejection of the state apparatus, the belief that human nature allows humans to exist in or progress toward such a non-coercive society and a suggestion on how to act to pursue the ideal of anarchy.{{sfn|McLaughlin|2007|pp=25–26}}

[[Herbert L. Osgood]] claimed that anarchism is "the extreme antithesis" of [[authoritarian communism]] and [[state socialism]].{{sfn|Osgood|1889|p=1}} [[Peter Marshall (author)|Peter Marshall]] states that "[i]n general anarchism is closer to socialism than liberalism. [...] Anarchism finds itself largely in the socialist camp, but it also has outriders in liberalism. It cannot be reduced to socialism, and is best seen as a separate and distinctive doctrine".{{sfn|Marshall|1992|p=641}} According to [[Jeremy Jennings]], "[i]t is hard not to conclude that these ideas", referring to [[anarcho-capitalism]], "are described as anarchist only on the basis of a misunderstanding of what anarchism is". Jennings adds that "anarchism does not stand for the untrammelled freedom of the individual (as the 'anarcho-capitalists' appear to believe) but, as we have already seen, for the extension of individuality and community".{{sfn|Jennings|1999|p=147}} [[Nicolas Walter]] wrote that "anarchism does derive from liberalism and socialism both historically and ideologically. [...] In a sense, anarchists always remain liberals and socialists, and whenever they reject what is good in either they betray anarchism itself. [...] We are liberals but more so, and socialists but more so".{{sfn|Walter|2002|p=44}} Michael Newman includes anarchism as one of many [[socialist traditions]], especially the more socialist-aligned tradition following Proudhon and [[Mikhail Bakunin]].{{sfn|Newman|2005|p=15}} [[Brian Morris (anthropologist)|Brian Morriss]] argues that it is "conceptually and historically misleading" to "create a dichotomy between socialism and anarchism".{{sfn|Morriss|2015|p=64}}

== History ==
{{main|History of anarchism}}

=== Pre-modern era ===
[[File:Paolo Monti - Servizio fotografico (Napoli, 1969) - BEIC 6353768.jpg|thumb|upright=.7|[[Zeno of Citium]] (c. 334 – c. 262 BC), whose ''[[Republic (Zeno)|Republic]]'' inspired [[Peter Kropotkin]]{{sfn|Marshall|1993|p=70}}]]
During the prehistoric era of mankind, an established authority did not exist. It was after the creation of towns and cities that institutions of authority were established and anarchistic ideas espoused as a reaction.{{sfn|Graham|2005|pp=xi–xiv}} Most notable precursors to anarchism in the ancient world were in China and Greece. In China, [[philosophical anarchism]] (i.e. the discussion on the legitimacy of the state) was delineated by [[Taoist]] philosophers [[Zhuang Zhou]] and [[Laozi]].{{sfnm|1a1=Coutinho|1y=2016|2a1=Marshall|2y=1993|2p=54}} Alongside [[Stoicism]], Taoism has been said to have had "significant anticipations" of anarchism.{{Sfn|Sylvan|2007|p=257}}

Anarchic attitudes were also articulated by tragedians and philosophers in Greece. [[Aeschylus]] and [[Sophocles]] used the myth of [[Antigone]] to illustrate the conflict between rules set by the state and personal [[autonomy]]. [[Socrates]] questioned Athenian authorities constantly and insisted on the right of individual freedom of conscience. [[Cynicism (philosophy)|Cynics]] dismissed human law (''[[Nomos (sociology)|nomos]]'') and associated authorities while trying to live according to nature (''[[physis]]''). [[Stoics]] were supportive of a society based on unofficial and friendly relations among its citizens without the presence of a state.{{sfn|Marshall|1993|pp=4, 66–73}}

During the [[Middle Ages]], there was no anarchistic activity except some ascetic religious movements in the [[Muslim world]] or in Christian Europe. This kind of tradition later gave birth to [[religious anarchism]]. In the [[Sasanian Empire]], [[Mazdak]] called for an [[egalitarian]] society and the [[abolition of monarchy]], only to be soon executed by Emperor [[Kavad I]].{{sfn|Marshall|1993|p=86}}

In [[Basra]], religious sects preached against the state. In Europe, various sects developed anti-state and libertarian tendencies. Libertarian ideas further emerged during the [[Renaissance]] with the spread of [[humanism]], [[rationalism]] and [[reason]]ing through Europe. Novelists fictionalised ideal societies that were based on voluntarism rather than coercion. The [[Age of Enlightenment]] further pushed towards anarchism with the optimism for social progress.{{sfn|Adams|2014|pp=33–63}}

=== Modern era ===
During the [[French Revolution]], partisan groups such as the [[Enragés]] and the {{lang|fr|[[sans-culottes]]}} saw a turning point in the fermentation of anti-state and federalist sentiments.{{sfn|Marshall|1993|p=4}} The first anarchist currents developed throughout the 18th century as [[William Godwin]] espoused [[philosophical anarchism]] in England, morally delegitimising the state, [[Max Stirner]]'s thinking paved the way to [[Individualist anarchism|individualism]] and [[Pierre-Joseph Proudhon]]'s theory of [[Mutualism (economic theory)|mutualism]] found fertile soil in France.{{sfn|Marshall|1993|pp=4–5}} By the late 1870s, various anarchist schools of thought had become well-defined and a wave of then unprecedented [[globalization]] occurred from 1880 to 1914.{{sfn|Levy|2011|pp=10–15}} This era of [[classical anarchism]] lasted until the end of the [[Spanish Civil War]] and is considered the golden age of anarchism.{{sfn|Marshall|1993|pp=4–5}}

[[File:Bakunin.png|thumb|upright|Mikhail Bakunin, who opposed the Marxist aim of [[dictatorship of the proletariat]] and allied himself with the federalists in the First International before his expulsion by the Marxists]]
Drawing from mutualism, [[Mikhail Bakunin]] founded [[collectivist anarchism]] and entered the [[International Workingmen's Association]], a class worker union later known as the First International that formed in 1864 to unite diverse revolutionary currents. The International became a significant political force, with [[Karl Marx]] being a leading figure and a member of its General Council. Bakunin's faction (the [[Jura Federation]]) and Proudhon's followers (the mutualists) opposed [[state socialism]], advocating political [[abstentionism]] and small property holdings.{{sfnm|1a1=Dodson|1y=2002|1p=312|2a1=Thomas|2y=1985|2p=187|3a1=Chaliand|3a2=Blin|3y=2007|3p=116}} After bitter disputes, the Bakuninists were expelled from the International by the [[Marxists]] at the [[1872 Hague Congress]].{{sfnm|1a1=Graham|1y=2019|1pp=334–336|2a1=Marshall|2y=1993|2p=24}} Anarchists were treated similarly in the [[Second International]], being ultimately expelled in 1896.{{sfn|Levy|2011|p=12}} Bakunin famously predicted that if revolutionaries gained power by Marx's terms, they would end up the new tyrants of workers. In response to their expulsion from the First International, anarchists formed the [[St. Imier International]]. Under the influence of [[Peter Kropotkin]], a Russian philosopher and scientist, [[anarcho-communism]] overlapped with collectivism.{{sfn|Marshall|1993|p=5}} Anarcho-communists, who drew inspiration from the 1871 [[Paris Commune]], advocated for free federation and for the distribution of goods according to one's needs.{{sfn|Graham|2005|p=xii}}

At the turn of the century, anarchism had spread all over the world.{{sfn|Moya|2015|p=327}} It was a notable feature of the international syndicalism movement.{{Sfn|Levy|2011|p=16}} In China, small groups of students imported the humanistic pro-science version of anarcho-communism.{{sfn|Marshall|1993|pp=519–521}} Tokyo was a hotspot for rebellious youth from countries of the far east, travelling to the Japanese capital to study.{{sfnm|1a1=Dirlik|1y=1991|1p=133|2a1=Ramnath|2y=2019|2pp=681–682}} In Latin America, [[Anarchism in Argentina|Argentina]] was a stronghold for [[anarcho-syndicalism]], where it became the most prominent left-wing ideology.{{sfnm|1a1=Levy|1y=2011|1p=23|2a1=Laursen|2y=2019|2p=157|3a1=Marshall|3y=1993|3pp=504–508}} During this time, a minority of anarchists adopted tactics of revolutionary [[political violence]]. This strategy became known as [[propaganda of the deed]].{{sfn|Marshall|1993|pp=633–636}} The dismemberment of the French socialist movement into many groups and the execution and exile of many [[Communards]] to penal colonies following the suppression of the Paris Commune favoured individualist political expression and acts.{{sfn|Anderson|2004}} Even though many anarchists distanced themselves from these terrorist acts, infamy came upon the movement and attempts were made to exclude them from American immigration, including the [[Immigration Act of 1903]], also called the Anarchist Exclusion Act.{{sfnm|1a1=Marshall|1y=1993|1pp=633–636|2a1=Lutz|2a2=Ulmschneider|2y=2019|2p=46}} [[Illegalism]] was another strategy which some anarchists adopted during this period.{{sfn|Bantman|2019|p=374}}

[[File:Makhno group.jpg|thumb|left|Nestor Makhno with members of the anarchist [[Revolutionary Insurrectionary Army of Ukraine]]]]
Despite concerns, anarchists enthusiastically participated in the [[Russian Revolution]] in opposition to the [[White movement]]. However, they met harsh suppression after the [[Bolshevik government]] was stabilized. Several anarchists from Petrograd and Moscow fled to Ukraine,{{sfn|Avrich|2006|p=204}} notably leading to the [[Kronstadt rebellion]] and [[Nestor Makhno]]'s struggle in the [[Free Territory]]. With the anarchists being crushed in Russia, two new antithetical currents emerged, namely [[platformism]] and [[synthesis anarchism]]. The former sought to create a coherent group that would push for revolution while the latter were against anything that would resemble a political party. Seeing the victories of the [[Bolsheviks]] in the [[October Revolution]] and the resulting [[Russian Civil War]], many workers and activists turned to [[communist parties]] which grew at the expense of anarchism and other socialist movements. In France and the United States, members of major syndicalist movements such as the [[General Confederation of Labour (France)|General Confederation of Labour]] and the [[Industrial Workers of the World]] left their organisations and joined the [[Communist International]].{{sfn|Nomad|1966|p=88}}

In the Spanish Civil War of 1936, anarchists and syndicalists ([[Confederación Nacional del Trabajo|CNT]] and [[Federación Anarquista Ibérica|FAI]]) once again allied themselves with various currents of leftists. A long tradition of [[Spanish anarchism]] led to anarchists playing a pivotal role in the war. In response to the army rebellion, an anarchist-inspired movement of peasants and workers, supported by armed militias, took control of Barcelona and of large areas of rural Spain, where they [[collectivised]] the land.{{sfn|Bolloten|1984|p=1107}} The [[Soviet Union]] provided some limited assistance at the beginning of the war, but the result was a bitter fight among communists and anarchists at a series of events named [[May Days]] as [[Joseph Stalin]] tried to seize control of the [[Republican faction (Spanish Civil War)|Republicans]].{{sfn|Marshall|1993|pp=xi, 466}}

=== Post-war era ===
[[File:Rojava Sewing Cooperative.jpg|thumb|Rojava's support efforts for workers to form cooperatives is exemplified in this sewing cooperative]]
At the end of [[World War II]], the anarchist movement was severely weakened.{{sfn|Marshall|1993|p=xi}} However, the 1960s witnessed a revival of anarchism, likely caused by a perceived failure of [[Marxism–Leninism]] and tensions built by the [[Cold War]].{{sfn|Marshall|1993|p=539}} During this time, anarchism found a presence in other movements critical towards both capitalism and the state such as the [[Anti-nuclear movement|anti-nuclear]], [[Environmental movement|environmental]] and [[peace movement]]s, the [[counterculture of the 1960s]] and the [[New Left]].{{sfn|Marshall|1993|pp=xi, 539}} It also saw a transition from its previous revolutionary nature to provocative [[anti-capitalist reform]]ism.{{sfn|Levy|2011|pp=5|p=}} Anarchism became associated with [[punk subculture]] as exemplified by bands such as [[Crass]] and the [[Sex Pistols]].{{sfn|Marshall|1993|pp=493–494}} The established [[feminist]] tendencies of [[anarcha-feminism]] returned with vigour during the [[second wave of feminism]].{{sfn|Marshall|1993|pp=556–557}} [[Black anarchism]] began to take form at this time and influenced anarchism's move from a [[Eurocentric]] demographic.{{sfn|Williams|2015|p=680}} This coincided with its failure to gain traction in Northern Europe and its unprecedented height in Latin America.{{sfn|Harmon|2011|p=70}}

Around the turn of the 21st century, anarchism grew in popularity and influence within [[anti-capitalist]], [[anti-war]] and [[anti-globalisation]] movements.{{sfn|Rupert|2006|p=66}} Anarchists became known for their involvement in protests against the [[World Trade Organization]] (WTO), the [[Group of Eight]] and the [[World Economic Forum]]. During the protests, ''ad hoc'' leaderless anonymous cadres known as [[black bloc]]s engaged in [[riot]]ing, [[property destruction]] and violent confrontations with the [[police]]. Other organisational tactics pioneered in this time include [[affinity group]]s, [[security culture]] and the use of decentralised technologies such as the Internet. A significant event of this period was the confrontations at the [[1999 Seattle WTO conference]].{{sfn|Rupert|2006|p=66}} Anarchist ideas have been influential in the development of the [[Zapatista Army of National Liberation|Zapatistas]] in Mexico and the Democratic Federation of Northern Syria, more commonly known as [[Rojava]], a ''de facto'' [[Permanent autonomous zone|autonomous region]] in northern Syria.{{sfn|Ramnath|2019|p=691}}

== Thought ==
{{main|Anarchist schools of thought}}
Anarchist schools of thought have been generally grouped into two main historical traditions, [[social anarchism]] and [[individualist anarchism]], owing to their different origins, values and evolution.{{sfnm|1a1=McLean|1a2=McMillan|1y=2003|1loc="Anarchism"|2a1=Ostergaard|2y=2003|2p=14|2loc="Anarchism"}} The individualist current emphasises [[negative liberty]] in opposing restraints upon the free individual while the social current emphasises [[positive liberty]] in aiming to achieve the free potential of society through equality and [[social ownership]].{{sfn|Harrison|Boyd|2003|p=251}} In a chronological sense, anarchism can be segmented by the classical currents of the late 19th century and the post-classical currents ([[anarcha-feminism]], [[green anarchism]] and [[post-anarchism]]) developed thereafter.{{sfn|Adams|Levy|2019|p=9}}

Beyond the specific factions of anarchist movements which constitute political anarchism lies [[philosophical anarchism]] which holds that the state lacks moral legitimacy, without necessarily accepting the imperative of revolution to eliminate it.{{sfn|Egoumenides|2014|p=2}} A component especially of individualist anarchism,{{sfnm|1a1=Ostergaard|1y=2003|1p=12|2a1=Gabardi|2y=1986|2pp=300–302}} philosophical anarchism may tolerate the existence of a [[minimal state]], but it argues that citizens have no [[moral obligation]] to obey government when it conflicts with individual autonomy.{{sfn|Klosko|2005|p=4}} Anarchism pays significant attention to moral arguments since [[ethics]] have a central role in anarchist philosophy.{{sfn|Franks|2019|p=549}} Anarchism's emphasis on [[anti-capitalism]], [[egalitarianism]] and for the extension of community and individuality sets it apart from [[anarcho-capitalism]] and other types of [[economic libertarian]]ism.{{sfnm|1a1=Marshall|1y=1992|1pp=564–565|2a1=Jennings|2y=1993|2p=143|3a1=Gay|3a2=Gay|3y=1999|3p=15|4a1=Morriss|4y=2008|4p=13|5a1=Johnson|5y=2008|5p=169|6a1=Franks|6y=2013|6pp=393–394}}

Anarchism is usually placed on the [[far-left]] of the [[political spectrum]].{{sfnm|1a1=Brooks|1y=1994|1p=xi|2a1=Kahn|2y=2000|3a1=Moynihan|3y=2007}} Much of its [[Anarchist economics|economics]] and [[Anarchist law|legal philosophy]] reflect [[anti-authoritarian]], [[anti-statist]], [[libertarian]] and [[Radical politics|radical]] interpretations of [[left-wing]] and [[socialist]] politics{{sfn|Guérin|1970|p=12}} such as [[Collectivist anarchism|collectivism]], [[Anarcho-communism|communism]], [[Individualist anarchism|individualism]], [[Mutualism (economic theory)|mutualism]] and [[Anarcho-syndicalism|syndicalism]], among other [[libertarian socialist]] economic theories.{{sfn|Guérin|1970|p=35|loc="Critique of authoritarian socialism"}} As anarchism does not offer a fixed body of doctrine from a single particular worldview,{{sfn|Marshall|1993|pp=14–17}} many anarchist types and traditions exist and varieties of anarchy diverge widely.{{sfn|Sylvan|2007|p=262}} One reaction against [[sectarianism]] within the anarchist milieu was [[anarchism without adjectives]], a call for toleration and unity among anarchists first adopted by [[Fernando Tarrida del Mármol]] in 1889 in response to the bitter debates of anarchist theory at the time.{{sfn|Avrich|1996|p=6}} Belief in political [[nihilism]] has been espoused by anarchists.{{sfn|Walter|2002|p=52}} Despite separation, the various anarchist schools of thought are not seen as distinct entities, but rather as tendencies that intermingle and are connected through a set of uniform principles such as individual and local autonomy, mutual aid, network organisation, communal democracy, justified authoritiy and decentralisation.{{sfnm|1a1=Marshall|1y=1993|1p=1–6|2a1=Angelbeck|2a2=Grier|2y=2012|2p=551}}

=== Classical ===
[[File:Portrait of Pierre Joseph Proudhon 1865.jpg|thumb|upright|Pierre-Joseph Proudhon, the primary proponent of mutualism, who influenced many future individualist anarchist and social anarchist thinkers{{sfn|Wilbur|2019|p=216–218}}]]
Inceptive currents among classical anarchist currents were [[Mutualism (economic theory)|mutualism]] and [[Individualist anarchism|individualism]]. They were followed by the major currents of [[social anarchism]] ([[Collectivist anarchism|collectivist]], [[Anarcho-communism|communist]] and [[Anarcho-syndicalism|syndicalist]]). They differ on organisational and economic aspects of their ideal society.{{sfn|Adams|Levy|2019|p=2}}

Mutualism is an 18th-century economic theory that was developed into anarchist theory by [[Pierre-Joseph Proudhon]]. Its aims include [[Reciprocity (cultural anthropology)|reciprocity]], [[Free association (Marxism and anarchism)|free association]], voluntary [[contract]], [[federation]] and [[monetary reform]] of both credit and currency that would be regulated by a bank of the people.{{sfn|Wilbur|2019|pp=213–218}} Mutualism has been retrospectively characterised as ideologically situated between individualist and collectivist forms of anarchism.{{sfnm|1a1=Avrich|1y=1996|1p=6|2a1=Miller|2y=1991|2p=11}} In ''[[What Is Property?]]'' (1840), Proudhon first characterised his goal as a "third form of society, the synthesis of communism and property".{{sfn|Pierson|2013|p=187}} Collectivist anarchism is a [[revolutionary socialist]] form of anarchism{{sfn|Morris|1993|p=76}} commonly associated with [[Mikhail Bakunin]].{{sfn|Shannon|2019|p=101}} Collectivist anarchists advocate [[collective ownership]] of the [[means of production]] which is theorised to be achieved through violent revolution{{sfn|Avrich|1996|pp=3–4}} and that workers be paid according to time worked, rather than goods being distributed according to need as in communism. Collectivist anarchism arose alongside [[Marxism]], but it rejected the [[dictatorship of the proletariat]] despite the stated Marxist goal of a collectivist [[stateless society]].{{sfnm|1a1=Heywood|1y=2017|1pp=146–147|2a1=Bakunin|2y=1990}}

[[Anarcho-communism]] is a theory of anarchism that advocates a [[communist society]] with [[common ownership]] of the means of production,{{sfn|Mayne|1999|p=131}} [[direct democracy]] and a [[Horizontalidad|horizontal]] network of [[voluntary association]]s, [[workers' council]]s and [[worker cooperative]]s, with production and consumption based on the guiding principle "[[From each according to his ability, to each according to his need]]".{{sfnm|1a1=Marshall|1y=1993|1p=327|2a1=Turcato|2y=2019|2pp=237–323}} Anarcho-communism developed from radical socialist currents after the [[French Revolution]],{{sfn|Graham|2005}} but it was first formulated as such in the Italian section of the [[First International]].{{sfn|Pernicone|2009|pp=111–113}} It was later expanded upon in the theoretical work of [[Peter Kropotkin]],{{sfn|Turcato|2019|pp=239–244}} whose specific style would go onto become the dominating view of anarchists by the late 19th century.{{sfn|Levy|2011|pp=6|p=}} Anarcho-syndicalism is a branch of anarchism that views [[labour syndicate]]s as a potential force for revolutionary social change, replacing capitalism and the state with a new society democratically self-managed by workers. The basic principles of anarcho-syndicalism are [[direct action]], workers' [[solidarity]] and [[workers' self-management]].{{sfn|Van der Walt|2019|p=249}}

Individualist anarchism is a set of several traditions of thought within the anarchist movement that emphasise the [[individual]] and their [[Will (philosophy)|will]] over any kinds of external determinants.{{sfn|Ryley|2019|p=225}} Early influences on individualist forms of anarchism include [[William Godwin]], [[Max Stirner]] and [[Henry David Thoreau]]. Through many countries, individualist anarchism attracted a small yet diverse following of Bohemian artists and intellectuals{{sfn|Marshall|1993|p=440}} as well as young anarchist outlaws in what became known as [[illegalism]] and [[individual reclamation]].{{sfnm|1a1=Imrie|1y=1994|2a1=Parry|2y=1987|2p=15}}

=== Post-classical and contemporary ===
{{main|Contemporary anarchism}}
[[File:Jarach and Zerzan.JPG|thumb|[[Lawrence Jarach]] (left) and [[John Zerzan]] (right), two prominent contemporary anarchist authors, with Zerzan being a prominent voice within [[anarcho-primitivism]] and Jarach a noted advocate of [[post-left anarchy]]]]
Anarchist principles undergird contemporary radical social movements of the left. Interest in the anarchist movement developed alongside momentum in the [[anti-globalisation movement]],{{sfn|Evren|2011|p=1}} whose leading activist networks were anarchist in orientation.{{sfn|Evren|2011|p=2}} As the movement shaped 21st century radicalism, wider embrace of anarchist principles signaled a revival of interest.{{sfn|Evren|2011|p=2}} Anarchism has continued to generate many philosophies and movements, at times eclectic, drawing upon various sources and [[syncretic]], combining disparate concepts to create new philosophical approaches.{{sfn|Perlin|1979}} The [[anti-capitalist]] tradition of classical anarchism has remained prominent within contemporary currents.{{sfn|Williams|2018|p=4}}

Contemporary news coverage which emphasizes [[black bloc]] demonstrations has reinforced anarchism's historical association with chaos and violence. However, its publicity has also led more scholars in fields such as [[anthropology]] and [[history]] to engage with the anarchist movement, although contemporary anarchism favours actions over academic theory.{{sfnm|1a1=Williams|1y=2010|1p=110|2a1=Evren|2y=2011|2p=1|3a1=Angelbeck|3a2=Grier|3y=2012|3p=549}} Various anarchist groups, tendencies and schools of thought exist today, making it difficult to describe the contemporary anarchist movement.{{sfn|Franks|2013|pp=385–386}} While theorists and activists have established "relatively stable constellations of anarchist principles", there is no consensus on which principles are core and commentators describe multiple "anarchisms" (rather than a singular "anarchism") in which common principles are shared between schools of anarchism while each group prioritizes those principles differently. Gender equality can be a common principle, although it ranks as a higher priority to [[anarcha-feminists]] than [[anarcho-communists]].{{sfn|Franks|2013|p=386}}

Anarchists are generally committed against coercive authority in all forms, namely "all centralized and hierarchical forms of government (e.g., monarchy, representative democracy, state socialism, etc.), economic class systems (e.g., capitalism, Bolshevism, feudalism, slavery, etc.), autocratic religions (e.g., fundamentalist Islam, Roman Catholicism, etc.), patriarchy, heterosexism, white supremacy, and imperialism".{{sfn|Jun|2009|pp=507–508}} However, anarchist schools disagree on the methods by which these forms should be opposed.{{sfn|Jun|2009|p=507}} The principle of [[equal liberty]] is closer to anarchist political ethics in that it transcends both the liberal and socialist traditions. This entails that liberty and equality cannot be implemented within the state, resulting in the questioning of all forms of domination and hierarchy.{{sfn|Egoumenides|2014|p=91}}

== Tactics ==
Anarchists' tactics take various forms, but in general, they serve two major goals, namely to first oppose [[the Establishment]] and secondly to promote anarchist ethics and reflect an anarchist vision of society, illustrating the unity of means and ends.{{sfn|Williams|2019|pp=107–108}} A broad categorisation can be made between aims to destroy oppressive states and institutions by revolutionary means on one hand and aims to change society through evolutionary means on the other.{{sfn|Williams|2018|pp=4–5}} Evolutionary tactics embrace [[nonviolence]], reject violence and take a gradual approach to anarchist aims, although there is significant overlap between the two.{{sfn|Kinna|2019|p=125}}

Anarchist tactics have shifted during the course of the last century. Anarchists during the early 20th century focused more on strikes and militancy while contemporary anarchists use a broader array of approaches.{{sfn|Williams|2019|p=112}}

=== Classical era tactics ===
[[File:McKinleyAssassination.jpg|thumb|The relationship between [[anarchism and violence]] is a controversial subject among anarchists as shown by anarchist [[Leon Czolgosz]] [[Assassination of William McKinley|assassinating]] [[William McKinley]]]]
During the classical era, anarchists had a militant tendency. Not only did they confront state armed forces, as in Spain and Ukraine, but some of them also employed [[terrorism]] as [[propaganda of the deed]]. Assassination attempts were carried out against heads of state, some of which were successful. Anarchists also took part in [[revolution]]s.{{sfn|Williams|2019|pp=112–113}} Many anarchists, especially the [[Galleanists]], believed that these attempts would be the impetus for a revolution against capitalism and the state.{{sfn|Norris|2020|pp=7-8}} Many of these attacks were done by individual assailants and the majority took place in the late 1870s, the early 1880s and the 1890s, with some still occurring in the early 1900s.{{sfnm|1a1=Levy|1y=2011|1p=13|2a1=Nesser|2y=2012|2p=62}} Their decrease in prevalence was the result of further [[judicial]] power and targeting and cataloging by state institutions.{{sfn|Harmon|2011|p=55}}

Anarchist perspectives towards violence have always been perplexing and controversial.{{sfn|Carter|1978|p=320}} On one hand, [[anarcho-pacifists]] point out the unity of means and ends.{{sfn|Fiala|2017}} On the other hand, other anarchist groups advocate direct action, a tactic which can include acts of [[sabotage]] or even acts of terrorism. This attitude was quite prominent a century ago when seeing the state as a [[tyrant]] and some anarchists believing that they had every right to oppose its [[oppression]] by any means possible.{{sfn|Kinna|2019|pp=116–117}} [[Emma Goldman]] and [[Errico Malatesta]], who were proponents of limited use of violence, argued that violence is merely a reaction to state violence as a [[necessary evil]].{{sfn|Carter|1978|pp=320–325}}

Anarchists took an active role in [[strike action]]s, although they tended to be antipathetic to formal [[syndicalism]], seeing it as [[Reformism|reformist]]. They saw it as a part of the movement which sought to overthrow the [[State (polity)|state]] and [[capitalism]].{{sfn|Williams|2019|p=113}} Anarchists also reinforced their propaganda within the arts, some of whom practiced [[naturism]] and [[nudism]]. Those anarchists also built communities which were based on [[friendship]] and were involved in the [[news media]].{{sfn|Williams|2019|p=114}}

=== Revolutionary tactics ===
[[File:Black bloc at RNC running.jpg|thumb|Black bloc protesters parading anarcho-communism imagery such as the motto "[[No War but the Class War]]"]]
In the current era, Italian anarchist [[Alfredo Bonanno]], a proponent of [[insurrectionary anarchism]], has reinstated the debate on violence by rejecting the nonviolence tactic adopted since the late 19th century by Kropotkin and other prominent anarchists afterwards. Both Bonanno and the French group [[The Invisible Committee]] advocate for small, informal affiliation groups, where each member is responsible for their own actions but works together to bring down oppression utilizing sabotage and other violent means against state, capitalism and other enemies. Members of The Invisible Committee were arrested in 2008 on various charges, terrorism included.{{sfn|Kinna|2019|pp=134–135}}

Overall, contemporary anarchists are much less violent and militant than their ideological ancestors. They mostly engage in confronting the police during demonstrations and riots, especially in countries such as [[Anarchism in Canada|Canada]], [[Anarchism in Greece|Greece]] and [[Anarchism in Mexico|Mexico]]. Militant [[black bloc]] protest groups are known for clashing with the police.{{sfn|Williams|2019|p=115}} However, anarchists not only clash with state operators; they also engage in the struggle against fascists and racists, taking [[anti-fascist action]] and mobilizing to prevent hate rallies from happening.{{sfn|Williams|2019|p=117}}

=== Evolutionary tactics ===
Anarchists commonly employ [[direct action]]. This can take the form of disrupting and protesting against unjust [[hierarchy]], or the form of self-managing their lives through the creation of counter-institutions such as communes and non-hierarchical collectives.{{sfn|Williams|2018|pp=4–5}}  Decision-making is often handled in an anti-authoritarian way, with everyone having equal say in each decision, an approach known as [[horizontalism]].{{sfn|Williams|2019|pp=109–117}} Contemporary-era anarchists have been engaging with various [[grassroots]] movements that are more or less based on horizontalism, although not explicitly anarchist, respecting personal autonomy and participating in mass activism such as strikes and demonstrations. In contrast with the ''big-A anarchism'' of the classical era, the newly coined term ''small-a anarchism'' signals their tendency not to base their thoughts and actions on classical-era anarchism or to refer to [[classical anarchists]] such as [[Peter Kropotkin]] and [[Pierre-Joseph Proudhon]] to justify their opinions. Those anarchists would rather base their thought and praxis on their own experience which they will later theorize.{{sfn|Kinna|2019|pp=145–149}}

The decision-making process of small anarchist [[affinity group]]s plays a significant tactical role.{{sfn|Williams|2019|pp=109, 119}} Anarchists have employed various methods in order to build a rough consensus among members of their group without the need of a leader or a leading group. One way is for an individual from the group to play the role of facilitator to help achieve a consensus without taking part in the discussion themselves or promoting a specific point. Minorities usually accept rough consensus, except when they feel the proposal contradicts anarchist ethics, goals and values. Anarchists usually form small groups (5–20 individuals) to enhance autonomy and friendships among their members. These kinds of groups more often than not interconnect with each other, forming larger networks. Anarchists still support and participate in strikes, especially [[Wildcat strike action|wildcat strikes]] as these are leaderless strikes not organised centrally by a syndicate.{{sfn|Williams|2019|p=119–121}}

As in the past, newspapers and journals are used, but anarchists have gone [[online]] in the [[World Wide Web]] to spread their message. However, anarchists have found it easier to create websites because of distributional and other difficulties, hosting electronic libraries and other portals.{{sfn|Williams|2019|pp=118–119}} Anarchists were also involved in developing various software that are available for free. The way these hacktivists work to develop and distribute resembles the anarchist ideals, especially when it comes to preserving users' privacy from state surveillance.{{sfn|Williams|2019|pp=120–121}}

Anarchists organize themselves to [[Squatting|squat]] and reclaim public spaces. During important events such as protests and when spaces are being occupied, they are often called [[Temporary Autonomous Zone]]s (TAZ), spaces where art, poetry and [[surrealism]] are blended to display the anarchist ideal.{{sfnm|1a1=Kinna|1y=2019|1p=139|2a1=Mattern|2y=2019|2p=596|3a1=Williams|3y=2018|3pp=5–6}} As seen by anarchists, squatting is a way to regain urban space from the capitalist market, serving pragmatical needs and also being an exemplary direct action.{{sfnm|1a1=Kinna|1y=2012|1p=250|2a1=Williams|2y=2019|2p=119}} Acquiring space enables anarchists to experiment with their ideas and build social bonds.{{sfn|Williams|2019|p=122}} Adding up these tactics while having in mind that not all anarchists share the same attitudes towards them, along with various forms of protesting at highly symbolic events, make up a [[carnivalesque]] atmosphere that is part of contemporary anarchist vividity.{{sfn|Morland|2004|p=37–38}}

== Key issues ==
{{main|Issues in anarchism}}
<!-- In the interest of restricting article length, please limit this section to two or three short paragraphs and add any substantial information to the main Issues in anarchism article. Thank you. -->
As anarchism is a [[philosophy]] that embodies many diverse attitudes, tendencies, schools of thought, disagreement over questions of values, ideology and tactics is common. Its diversity has led to widely different uses of identical terms among different anarchist traditions which has created a number of [[definitional concerns in anarchist theory]]. The compatibility of [[Anarchism and capitalism|capitalism]],{{sfnm|1a1=Marshall|1y=1993|1p=565|2a1=Honderich|2y=1995|2p=31|3a1=Meltzer|3y=2000|3p=50|4a1=Goodway|4y=2006|4p=4|5a1=Newman|5y=2010|5p=53}} [[Anarchism and nationalism|nationalism]] and [[Anarchism and religion|religion]] with anarchism is widely disputed. Similarly, anarchism enjoys complex relationships with ideologies such as [[Anarchism and communism|communism]], [[collectivism]], [[Anarchism and Marxism|Marxism]] and [[trade unionism]]. Anarchists may be motivated by [[humanism]], [[God|divine authority]], [[enlightened self-interest]], [[Veganarchism|veganism]], or any number of alternative ethical doctrines. Phenomena such as [[civilisation]], [[technology]] (e.g. within [[anarcho-primitivism]]) and the [[Anarchism and statist democracy|democratic process]] may be sharply criticised within some anarchist tendencies and simultaneously lauded in others.{{sfn|De George|2005|pp=31–32}}

=== Gender, sexuality and free love ===
{{see also|Anarchism and issues related to love and sex}}
[[File:Emilearmand01.jpg|left|thumb|upright|[[Émile Armand]], a French individualist anarchist who propounded the virtues of free love in the Parisian anarchist milieu of the early 20th century]]
As gender and sexuality carry along them dynamics of hierarchy, anarchism is obliged to address, analyse and oppose the suppression of one's autonomy because of the dynamics that gender roles traditionally impose.{{sfn|Nicholas|2019|p=603}}

[[File:AnarchaFeminism.png|thumb|upright=1.2|Collection of [[anarcha-feminist]] protests, symbols and flags]]
Sexuality was not often discussed by classical anarchists, but the few that did felt that an anarchist society would lead to sexuality naturally developing.{{sfn|Lucy|2020|p=162}} However, sexual violence was a concern for anarchists such as [[Benjamin Tucker]], who opposed age of consent laws, believing they would benefit predatory men.{{sfn|Lucy|2020|p=178}} A historical current that arose and flourished during 1890 and 1920 within anarchism was [[free love]]. In contemporary anarchism, this current survives as a tendency to support [[polyamory]] and [[queer anarchism]].{{sfnm|1a1=Nicholas|1y=2019|1p=611|2a1=Jeppesen|2a2=Nazar|2y=2012|2pp=175–176}} Free love advocates were against marriage, which they saw as a way of men imposing authority over women, largely because marriage law greatly favoured the power of men. The notion of free love was much broader and included a critique of the established order that limited women's sexual freedom and pleasure.{{sfn|Jeppesen|Nazar|2012|pp=175–176}} Those free love movements contributed to the establishment of communal houses, where large groups of travelers, anarchists and other activists slept in beds together.{{sfn|Jeppesen|Nazar|2012|p=177}} Free love had roots both in Europe and the United States. However, some anarchists struggled with the jealousy that arose from free love.{{sfn|Jeppesen|Nazar|2012|pp=175–177}} Anarchist feminists were advocates of free love, against marriage, pro-choice (utilising a contemporary term) and had a similar agenda. Anarchist and non-anarchist feminists differed on [[suffrage]], but they were nonetheless supportive of one another.{{sfn|Kinna2019|pp=166–167}}

During the second half of the 20th century, anarchism intermingled with the [[second wave of feminism]], radicalising some currents of the feminist movement and being influenced as well. By the latest decades of the 20th century, anarchists and feminists were advocating for the rights and autonomy of women, gays, queers and other marginalised groups, with some feminist thinkers suggesting a fusion of the two currents.{{sfn|Nicholas|2019|pp=609–611}} With the [[third wave of feminism]], sexual identity and compulsory heterosexuality became a subject of study for anarchists, yielding a [[post-structuralist]] critique of sexual normality.{{sfn|Nicholas|2019|pp=610–611}} However, some anarchists distanced themselves from this line of thinking, suggesting that it leaned towards an individualism that was dropping the cause of social liberation.{{sfn|Nicholas|2019|pp=616–617}}

=== Anarchism and education ===
{{main|Anarchism and education}}
{|class="wikitable" style="border: none; background: none; float: right;"
|+ Anarchist vs. statist perspectives on education<br><small>Ruth Kinna (2019){{sfn|Kinna|2019|p=97}}</small>
|-
!scope="col"|
!scope="col"|Anarchist education
!scope="col"|State education
|-
|Concept || Education as self-mastery || Education as service
|-
|Management || Community based || State run
|-
|Methods || Practice-based learning || Vocational training
|-
|Aims || Being a critical member of society || Being a productive member of society
|}
The interest of anarchists in education stretches back to the first emergence of classical anarchism. Anarchists consider proper education, one which sets the foundations of the future autonomy of the individual and the society, to be an act of [[Mutual aid (organization theory)|mutual aid]].{{sfnm|1a1=Kinna|1y=2019|1pp=83–85|2a2=Suissa|2y=2019|2pp=514–515, 520}} Anarchist writers such as [[William Godwin]] (''[[Political Justice]]'') and [[Max Stirner]] ("[[The False Principle of Our Education]]") attacked both state education and private education as another means by which the ruling class replicate their privileges.{{sfnm|1a1=Suissa|1y=2019|1pp=514, 521|2a1=Kinna|2y=2019|2pp=83–86|3a1=Marshall|3y=1993|3p=222}}

In 1901, Catalan anarchist and free thinker [[Francisco Ferrer]] established the [[Escuela Moderna]] in Barcelona as an opposition to the established education system which was dictated largely by the Catholic Church.{{sfn|Suissa|2019|pp=511–512}} Ferrer's approach was secular, rejecting both state and church involvement in the educational process whilst giving pupils large amounts of autonomy in planning their work and attendance. Ferrer aimed to educate the working class and explicitly sought to foster [[class consciousness]] among students. The school closed after constant harassment by the state and Ferrer was later arrested. Nonetheless, his ideas formed the inspiration for a series of [[Modern School (United States)|modern schools]] around the world.{{sfn|Suissa|2019|pp=511–514}} Christian anarchist [[Leo Tolstoy]], who published the essay ''Education and Culture'', also established a similar school with its founding principle being that "for education to be effective it had to be free".{{sfn|Suissa|2019|pp=517–518}} In a similar token, A. S. Neill founded what became the [[Summerhill School]] in 1921, also declaring being free from coercion.{{sfn|Suissa|2019|pp=518–519}}

Anarchist education is based largely on the idea that a child's right to develop freely and without manipulation ought to be respected and that rationality will lead children to morally good conclusions. However, there has been little consensus among anarchist figures as to what constitutes manipulation. Ferrer believed that moral indoctrination was necessary and explicitly taught pupils that equality, liberty and social justice were not possible under capitalism, along with other critiques of government and nationalism.{{sfnm|1a1=Avrich|1y=1980|1pp=3–33|2a1=Suissa|2y=2019|2pp=519–522}}

Late 20th century and contemporary anarchist writers ([[Colin Ward]], [[Herbert Read]] and [[Paul Goodman]]) intensified and expanded the anarchist critique of state education, largely focusing on the need for a system that focuses on children's creativity rather than on their ability to attain a career or participate in [[consumerism]] as part of a consumer society.{{sfn|Kinna|2019|pp=89–96}} Contemporary anarchists such as Ward have further argued that state education serves to perpetuate socioeconomic inequality.{{sfn|Ward|1973|pp=39–48}}

While few anarchist education institutions have survived to the modern-day, major tenets of anarchist schools, among them respect for child autonomy and relying on reasoning rather than indoctrination as a teaching method, have spread among mainstream educational institutions. Judith Suissa names three schools as explicitly anarchists schools, namely the Free Skool Santa Cruz in the United States which is part of a wider American-Canadian network of schools, the Self-Managed Learning College in Brighton, England and the Paideia School in Spain.{{sfn|Suissa|2019|pp=523–526}}

=== Anarchism and the state ===
<!-- Important! Strive to explain how anarchists perceive authority and oppression and why they reject them. Jun (2019), p. 41. -->
Objection to the [[State (polity)|state]] and its institutions is a ''[[sine qua non]]'' of anarchism.{{sfnm|1a1=Carter|1y=1971|1p=14|2a1=Jun|2y=2019|2pp=29–30}} Anarchists consider the state as a tool of domination and believe it to be illegitimate regardless of its political tendencies. Instead of people being able to control the aspects of their life, major decisions are taken by a small elite. Authority ultimately rests solely on power, regardless of whether that power is [[Open government|open]] or [[Transparency (behavior)|transparent]], as it still has the ability to coerce people. Another anarchist argument against states is that the people constituting a government, even the most altruistic among officials, will unavoidably seek to gain more power, leading to corruption. Anarchists consider the idea that the state is the collective will of the people to be an unachievable fiction due to the fact that the [[ruling class]] is distinct from the rest of society.{{sfn|Jun|2019|pp=32–38}}

Specific anarchist attitudes towards the state vary. [[Robert Paul Wolff]] believed that the tension between authority and autonomy would mean the state could never be legitimate. Bakunin saw the state as meaning "coercion, domination by means of coercion, camouflaged if possible but unceremonious and overt if need be". [[A. John Simmons]] and [[Leslie Green (philosopher)|Leslie Green]], who leaned toward philosophical anarchism, believed that the state could be legitimate if it is governed by consensus, although they saw this as highly unlikely.{{sfnm|1a1=Wendt|1y=2020|1p=2|2a1=Ashwood|2y=2018|2p=727}} Beliefs on how to abolition the state also differ.{{Sfn|Ashwood|2018|p=735}}

=== Anarchism and art ===
{{see also|Anarchism and the arts}}
[[File:Apple Harvest by Camille Pissarro.jpg|thumb|340px|''Les chataigniers a Osny'' (1888) by anarchist painter Camille Pissarro, an example of blending anarchism and art]]
The connection between anarchism and art was quite profound during the classical era of anarchism, especially among artistic currents that were developing during that era such as futurists, surrealists and others.{{sfn|Mattern|2019|p=592}} In literature, anarchism was mostly associated with the [[New Apocalyptics]] and the [[neo-romanticism]] movement.{{snf|Gifford|2019|p=577}} In music, anarchism has been associated with music scenes such as punk.{{sfn|AllMusic}} Anarchists such as [[Leo Tolstoy]] and [[Herbert Read]] argued that the border between the artist and the non-artist, what separates art from a daily act, is a construct produced by the alienation caused by capitalism and it prevents humans from living a joyful life.{{sfn|Mattern|2019|pp=592–593}}

Other anarchists advocated for or used art as a means to achieve anarchist ends.{{sfn|Mattern|2019|p=593}} In his book ''Breaking the Spell: A History of Anarchist Filmmakers, Videotape Guerrillas, and Digital Ninjas'', Chris Robé claims that "anarchist-inflected practices have increasingly structured movement-based video activism".{{sfn|Robé|2017|p=44}} Throughout the 20th century, many prominent anarchists ([[Peter Kropotkin]], [[Emma Goldman]], [[Gustav Landauer]] and [[Camillo Berneri]]) and publications such as ''[[Anarchy (magazine)|Anarchy]]'' wrote about matters pertaining to the arts.{{sfn|Miller|Dirlik|Rosemont|Augustyn|Duignan|Lotha|2019|p=1}}

Three overlapping properties made art useful to anarchists. It could depict a critique of existing society and hierarchies, serve as a prefigurative tool to reflect the anarchist ideal society and even turn into a means of direct action such as in protests. As it appeals to both emotion and reason, art could appeal to the whole human and have a powerful effect.{{sfn|Mattern|2019|pp=593–596}} The 19th-century [[neo-impressionist]] movement had an ecological aesthetic and offered an example of an anarchist perception of the road towards socialism.{{sfn|Antliff|1998|p=78}} In ''Les chataigniers a Osny'' by anarchist painter [[Camille Pissarro]], the blending of aestetic and social harmony is prefiguring an ideal anarchistic agrarian community.{{sfn|Antliff|1998|p=99}}

== Criticism{{anchor|Criticisms}} ==
{{multiple image
|direction = vertical
|alignment = center
|perrow = 2
|total_width = 190
|image2 = Bertrand Russell 1957.jpg
|image1 = Karl Marx 001.jpg
|footer = [[Karl Marx]] (top) called anarchism "schoolboy asininity" while [[Bertrand Russell]] (bottom) argued that anarchism was unable to provide all the services that government does
}}
The most common critique of anarchism is that humans can not self govern and a state is necessary for human survival.{{sfn|Peterson|1987|p=238}} Philosopher [[Bertrand Russell]] supported this critique, noting that "[p]eace and war, tariffs, regulations of sanitary conditions and the sale of noxious drugs, the preservation of a just system of distribution: these, among others, are functions which could hardly be performed in a community in which there was no central government".{{sfn|Peterson|1987|p=238}} Another common criticism of anarchism is that it fits a world of isolation in which only the small enough entities can be self-governing. [[Colin Ward]] responds that major anarchist thinkers advocated [[Anarchist federalism|federalism]].{{sfn|Ward|2004|p=78}}

Philosophy lecturer Andrew G. Fiala also believed that humans could not self govern and included it in his list of arguments against anarchism. Fiala's other critiques were that anarchism is innately related to violence and destruction, not only in the pragmatic world, i.e. at protests, but in the world of ethics as well. Secondly, anarchism is evaluated as unfeasible or utopian since the state can not be defeated practically. This line of arguments most often calls for political action within the system to reform it. The third argument is that anarchism is self-contradictory. While it advocates for no-one to ''archiei'', if accepted by the many, then anarchism would turn into the ruling political theory. In this line of criticism also comes the self-contradiction that anarchism calls for collective action whilst endorsing the autonomy of the individual, hence no collective action can be taken. Lastly, Fiala mentions a critique towards philosophical anarchism of being ineffective (all talk and thoughts) and in the meantime capitalism and bourgeois class remains strong.{{sfn|Fiala|2017|loc="4. Objections and Replies"}}

Philosophical anarchism has met the criticism of members of academia following the release of pro-anarchist books such as [[A. John Simmons]]' ''Moral Principles and Political Obligations''.{{sfn|Klosko|1999|p=536}} Law professor William A. Edmundson authored an essay arguing against three major philosophical anarchist principles which he finds fallacious. Edmundson claims that while the individual does not owe the state a duty of obedience, this does not imply that anarchism is the inevitable conclusion and the state is still morally legitimate.{{sfnm|1a1=Klosko|1y=1999|1p=536|2a1=Kristjánsson|2y=2000|2p=896}} In ''The Problem of Political Authority'', [[Michael Huemer]] defends philosophical anarchism,{{sfn|Dagger|2018|p=35}} claiming that "political authority is a moral illusion".{{sfn|Rogers|2020|p=}}

Another criticism is that anarchism defies and fails to understand the biological inclination to authority as first articulated in an 1886 article for the ''[[North American Review]]'' by Frances L. Ferguson.{{sfn|Ferguson|1886}} [[Joseph Raz]] argues that the acceptance of authority implies the belief that following their instructions will afford more success.{{sfn|Gans|1992|p=37}} Raz believes that this argument is true in following both authorities' successful and mistaken instruction.{{sfn|Gans|1992|p=38}} Anarchists reject this criticism because challenging or disobeying authority does not entail the disappearance of its advantages by acknowledging authority such as doctors or lawyers as reliable, nor does it involve a complete surrender of independent judgment.{{sfn|Gans|1992|pp=34, 38}} Anarchist perception of human nature, rejection of the state and commitment to social revolution has been criticised by academics as naive, overly simplistic and unrealistic, respectively.{{sfn|Brinn|2020|p=206}} Classical anarchism has been criticised for relying too heavily on the belief that the abolition of the state will lead to human cooperation prospering.{{sfn|Lucy|2020|p=162}} 

Academic [[John Molyneux (academic)|John Molyneux]] notes in his book ''Anarchism: A Marxist Criticism'' that "anarchism cannot win", believing that it lacks the ability to properly implement its ideas.{{sfn|Dodds|2011}} The Marxist criticism of anarchism is that it has a utopian character because all individuals should have anarchist views and values. According to the Marxist view, that a social idea would follow directly from this human ideal and out of the free will of every individual formed its essence. Marxists argue that this contradiction was responsible for their inability to act. In the anarchist vision, the conflict between liberty and equality was resolved through coexistence and intertwining.{{sfn|Baár|Falina|Janowski|Kopeček|2016|p=488}} Anarchism is also seen by Marxists as unrealistic and overly idealistic. [[Karl Marx]] described it as "schoolboy asininity".{{sfn|Brinn|2020|p=206}}

== See also ==
{{Portal|Anarchism|Libertarianism}}
* [[:Category:Anarchism by country|Anarchism by country]]
* [[Governance without government]]
* [[List of anarchist political ideologies]]
* [[List of books about anarchism]]

== References ==
=== Citations ===
{{reflist|25em}}

=== Sources ===
==== Primary sources ====
{{refbegin|35em|indent=yes}}
* {{cite book|last=Bakunin|first=Mikhail|author-link=Mikhail Bakunin|title=Statism and Anarchy|title-link=Statism and Anarchy|year=1990|orig-year=1873|publisher=Cambridge University Press|location=Cambridge, England |series=Cambridge Texts in the History of Political Thought|translator-last=Shatz|translator-first=Marshall|isbn=978-0-521-36182-8|oclc=20826465|lccn=89077393|doi=10.1017/CBO9781139168083|editor1-last=Shatz|editor1-first=Marshall}}
{{refend}}

==== Secondary sources ====
{{refbegin|35em|indent=yes}}
* {{cite journal|last=Adams|first=Matthew S.|date=14 January 2014|title=The Possibilities of Anarchist History: Rethinking the Canon and Writing History|url=https://journals.uvic.ca/index.php/adcs/article/view/17138|journal=Anarchist Developments in Cultural Studies|volume=2013.1: Blasting the Canon|pages=33–63|accessdate=17 December 2019|via=University of Victoria Libraries}}
* {{cite book|editor-last1=Adams|editor-first1=Matthew S.|editor-last2=Levy|editor-first2=Carl|title=The Palgrave Handbook of Anarchism|year=2018|isbn=978-3-319-75619-6|publisher=Palgrave Macmillan}}
* {{cite book|last1=Adams|first1=Matthew S.|last2=Levy|first2=Carl|chapter=Introduction|pages=1–23|editor-last1=Adams|editor-first1=Matthew S.|editor-last2=Levy|editor-first2=Carl|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite journal|last1=Angelbeck|first1=Bill|last2=Grier|first2=Colin|year=2012|title=Anarchism and the Archaeology of Anarchic Societies: Resistance to Centralization in the Coast Salish Region of the Pacific Northwest Coast|journal=Current Anthropology|volume=53|issue=5|pages=547–587|doi=10.1086/667621}}
* {{cite journal|last=Antliff|first=Mark|year=1998|title=Cubism, Futurism, Anarchism: The 'Aestheticism' of the "Action d'art" Group, 1906–1920|journal=Oxford Art Journal|volume=21|issue=2|pages=101–120|jstor=1360616|doi=10.1093/oxartj/21.2.99}}
* {{cite journal|last=Anderson|first=Benedict|author-link=Benedict Anderson|title=In the World-Shadow of Bismarck and Nobel|journal=[[New Left Review]]|volume=2|issue=28|pages=85–129|year=2004|url=http://newleftreview.org/II/28/benedict-anderson-in-the-world-shadow-of-bismarck-and-nobel|access-date=7 January 2016|archive-url=https://web.archive.org/web/20151219130121/http://newleftreview.org/II/28/benedict-anderson-in-the-world-shadow-of-bismarck-and-nobel|archive-date=19 December 2015}}
* {{cite journal|last=Ashwood|first=Loka|year=2018|title=Rural Conservatism or Anarchism? The Pro‐state, Stateless, and Anti‐state Positions|journal=Rural Sociology|volume=83|issue=4|pages=717–748|doi=10.1111/ruso.12226}}
* {{cite book|last=Avrich|first=Paul|title=Anarchist Voices: An Oral History of Anarchism in America|year=1996|publisher=Princeton University Press|isbn=978-0-691-04494-1}}
* {{cite book|last=Avrich|first=Paul|author-link=Paul Avrich|title=The Russian Anarchists|publisher=[[AK Press]]|location=Stirling|year=2006|isbn=978-1-904859-48-2|title-link=The Russian Anarchists}}
* {{cite book|last=Avrich|first=Paul|url=https://books.google.com/books?id=X6X_AwAAQBAJ&pg=PA3|title=The Modern School Movement: Anarchism and Education in the United States|publisher=Princeton University Press|year=1980|isbn=978-1-4008-5318-2|pages=3–33|oclc=489692159}}
* {{cite book|last1=Baár|first1=Monika|last2=Falina|first2=Maria|last3=Janowski|first3=Maciej|last4=Kopeček|first4=Michal|last5=Trencsényi|first5=Balázs Trencsényi|year=2016|title=A History of Modern Political Thought in East Central Europe: Negotiating Modernity in the 'Long Nineteenth Century'|volume=I|location=Oxford|publisher=Oxford University Press|isbn=9780191056956}}
* {{cite book|last=Bantman|first=Constance|chapter=The Era of Propaganda by the Deed|pages=371–388|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Bates|first=David|editor=Paul Wetherly|title=Political Ideologies|chapter-url=https://books.google.com/books?id=uXfJDgAAQBAJ&pg=PA128|year=2017|publisher=Oxford University Press|isbn=978-0-19-872785-9|chapter=Anarchism}}
* {{cite journal|last=Bernstein|first=David Eliot|year=2020|title=The Right to Armed Self-Defense in the Light of Law Enforcement Abdication|publisher=George Mason University|journal=Liberty & Law Center|issue=20–23|pages=1–42}}
* {{cite book|last=Bolloten|first=Burnett|author-link=Burnett Bolloten|title=The Spanish Civil War: Revolution and Counterrevolution|publisher=University of North Carolina Press|year=1984|isbn=978-0-8078-1906-7}}
* {{cite journal|last=Brinn|first=Gearóid|year=2020|title=Smashing the State Gently: Radical Realism and Realist Anarchism|journal=European Journal of Political Theory|volume=19|issue=2|pages=206–227|doi=10.1177/1474885119865975|s2cid=202278143}}
* {{cite book|last=Brooks|first=Frank H.|year=1994|title=The Individualist Anarchists: An Anthology of Liberty (1881–1908)|publisher=Transaction Publishers|isbn=978-1-56000-132-4}}
* {{cite book|last=Carter|first=April|author-link=April Carter|title=The Political Theory of Anarchism|url=https://books.google.com/books?id=3mlWPgAACAAJ|year=1971|publisher=Routledge|isbn=978-0-415-55593-7}}
* {{cite journal|last=Carter|first=April|title=Anarchism and violence|journal=Nomos|volume=19|pages=320–340|year=1978|publisher=American Society for Political and Legal Philosophy|jstor=24219053}}
* {{cite book|editor1-last=Chaliand|editor1-first=Gerard|editor2-last=Blin|editor2-first=Arnaud|title=The History of Terrorism: From Antiquity to Al-Quaeda|publisher=University of California Press|location=Berkeley; Los Angeles; London|year=2007|isbn=978-0-520-24709-3|oclc=634891265|url-access=registration|url=https://archive.org/details/historyofterrori00grar}}
* {{cite book|last=Chomsky|first=Noam|author-link=Noam Chomsky|editor-last=Pateman|editor-first=Barry|title=Chomsky on Anarchism|year=2005|isbn=978-1-904859-26-0|publisher=AK Press|location=Oakland}}
* {{cite book|last=Cohn|first=Jesse|editor-last=Ness|editor-first=Immanuel|chapter=Anarchism|title=The International Encyclopedia of Revolution and Protest|pages=1–11|year=2009|isbn=978-1-4051-9807-3|publisher=John Wiley & Sons|location=Oxford|doi=10.1002/9781405198073.wbierp0039}}
* {{cite book|last=Dagger|first=Tristan J.|year=2018|title=Playing Fair: Political Obligation and the Problems of Punishment|location=Oxford|publisher=Oxford University Press|isbn=9780199388837}}
* {{cite book|last=Dirlik|first=Arif|title=Anarchism in the Chinese Revolution|publisher=University of California Press|location=Berkeley, CA|year=1991|isbn=978-0-520-07297-8}}
* {{cite magazine|last=Dodds|first=Jonathan|date=October 2011|url=http://socialistreview.org.uk/362/anarchism-marxist-criticism|title=Anarchism: A Marxist Criticism|magazine=Socialist Review|accessdate=31 July 2020}}
* {{cite book|last=Dodson|first=Edward|title=The Discovery of First Principles|volume=2|publisher=Authorhouse|year=2002|isbn=978-0-595-24912-1}}
* {{cite book|last=Egoumenides|first=Magda|year=2014|url=https://books.google.com/books?id=4qEMBAAAQBAJ|title=Philosophical Anarchism and Political Obligation|location=New York|publisher=Bloomsbury Publishing USA|isbn=978-1-4411-2445-6}}
* {{cite book|last=Evren|first=Süreyyya|author-link=Süreyyya Evren|chapter=How New Anarchism Changed the World (of Opposition) after Seattle and Gave Birth to Post-Anarchism|pages=1–19|editor-last1=Rousselle|editor-first1=Duane|editor-last2=Evren|editor-first2=Süreyyya|title=Post-Anarchism: A Reader|year=2011|isbn=978-0-7453-3086-0|publisher=[[Pluto Press]]}}
* {{cite journal|last=Ferguson|first=Francis L.|date=August 1886|title=The Mistakes of Anarchism|journal=The North American Review|publisher=University of Northern Iowa|volume=143|issue=357|pages=204–206|issn=0029-2397|jstor=25101094}}
* {{cite book|last=Fernández|first=Frank|title=Cuban Anarchism: The History of A Movement|year=2009|orig-year=2001|publisher=Sharp Press}}
* {{cite journal|last1=Franks|first1=Benjamin|author-link=Benjamin Franks|editor-last1=Freeden|editor-first1=Michael|editor-last2=Stears|editor-first2=Marc|title=Anarchism|journal=The Oxford Handbook of Political Ideologies|pages=385–404|date=August 2013|publisher=Oxford University Press|doi=10.1093/oxfordhb/9780199585977.013.0001}}
* {{cite book|last1=Franks|first1=Benjamin|author-link=Benjamin Franks|chapter=Anarchism and Ethics|pages=549–570|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite journal|last=Gabardi|first=Wayne|year=1986|volume=80|issue=1|pages=300–302|doi=10.2307/1957102|jstor=446800|title=Anarchism. By David Miller. (London: J. M. Dent and Sons, 1984. Pp. 216. £10.95.)|journal=American Political Science Review}}
* {{cite book|last=Gans|first=Chaim|year=1992|title=Philosophical Anarchism and Political Disobedience|edition=reprint|location=Cambridge|publisher=Cambridge University Press|isbn=9780521414500}}
* {{cite book|last1=Gay|first1=Kathlyn|last2=Gay|first2=Martin|title=Encyclopedia of Political Anarchy|year=1999|isbn=978-0-87436-982-3|publisher=ABC-CLIO}}
* {{cite book|last=Gifford|first=James|chapter=Literature and Anarchism|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Goodway|first=David|author-link=David Goodway|title=Anarchist Seeds Beneath the Snow|publisher=Liverpool Press|year=2006|isbn=978-1-84631-025-6}}
* {{cite book|last=Graham|first=Robert|title=Anarchism: a Documentary History of Libertarian Ideas: from Anarchy to Anarchism|publisher=Black Rose Books|location=Montréal|year=2005|isbn=978-1-55164-250-5|author-link=Robert Graham (historian)|url=http://robertgraham.wordpress.com/anarchism-a-documentary-history-of-libertarian-ideas-volume-one-from-anarchy-to-anarchism-300ce-1939/|access-date=5 March 2011|archive-url=https://web.archive.org/web/20101130131904/http://robertgraham.wordpress.com/anarchism-a-documentary-history-of-libertarian-ideas-volume-one-from-anarchy-to-anarchism-300ce-1939/|archive-date=30 November 2010}}
* {{cite book|last=Graham|first=Robert|author-link=Robert Graham (historian)|chapter=Anarchism and the First International|pages=325–342|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|chapter-url=https://books.google.com/books?id=SRyQswEACAAJ|year=2019|publisher=Springer|isbn=978-3-319-75620-2}}
* {{cite book|last=Guérin|first=Daniel|author-link=Daniel Guérin|year=1970|url=http://theanarchistlibrary.org/library/daniel-guerin-anarchism-from-theory-to-practice|title=Anarchism: From Theory to Practice|publisher=Monthly Review Press|isbn=9780853451280}}
* {{cite book|last1=Harrison|first1=Kevin|last2=Boyd|first2=Tony|title=Understanding Political Ideas and Movements|url=https://books.google.com/books?id=5qrJCgAAQBAJ|date=5 December 2003|publisher=Manchester University Press|isbn=978-0-7190-6151-6}}
* {{cite journal|last=Harmon|first=Christopher C.|year=2011|title=How Terrorist Groups End: Studies of the Twentieth Century|journal=Connections|volume=10|issue=2|pages=51–104|jstor=26310649}}
* {{cite book|last=Heywood|first=Andrew|author-link=Andrew Heywood|year=2017|title=Political Ideologies: An Introduction|url=https://books.google.com/books?id=Sy8hDgAAQBAJ&pg=PA146|edition=6th|publisher=Macmillan International Higher Education|isbn=978-1-137-60604-4}}
* {{cite book|last=Honderich|first=Ted|title=The Oxford Companion to Philosophy|url=https://archive.org/details/oxfordcompaniont00hond|url-access=registration|year=1995|publisher=Oxford University Press|isbn=978-0-19-866132-0}}
* {{cite news|last=Imrie|first=Doug|title=The Illegalists|year=1994|work=Anarchy: A Journal of Desire Armed|url=http://recollectionbooks.com/siml/library/illegalistsDougImrie.htm|url-status=dead|archive-url=https://web.archive.org/web/20150908072801/http://recollectionbooks.com/siml/library/illegalistsDougImrie.htm|archive-date=8 September 2015|access-date=9 December 2010}}
* {{cite book|last1=Jennings|first1=Jeremy|editor-last1=Eatwell|editor-first1=Roger|editor-last2=Wright|editor-first2=Anthony|chapter=Anarchism|title=Contemporary Political Ideologies|pages=127–146|year=1993|isbn=978-0-86187-096-7|publisher=Pinter|location=London}}
* {{cite book|last=Jennings|first=Jeremy|year=1999|chapter=Anarchism|editor-last1=Eatwell|editor-first1=Roger|editor-last2=Wright|editor-first2=Anthony|title=Contemporary Political Ideologies|edition=reprinted, 2nd|location=London|publisher=A & C Black|isbn=9780826451736}}
* {{cite book|last1=Jeppesen|first1=Sandra|last2=Nazar|first2=Holly|chapter=Genders and Sexualities in Anarchist Movements|editor=[[Ruth Kinna]]|title=The Bloomsbury Companion to Anarchism|url=https://books.google.com/books?id=dNuoAwAAQBAJ|date=28 June 2012|publisher=Bloomsbury Publishing|isbn=978-1-4411-4270-2}}
* {{cite book|last1=Johnson|first1=Charles|editor-last1=Long|editor-first1=Roderick T.|editor-last2=Machan|editor-first2=Tibor R.|editor-link1=Roderick T. Long|editor-link2=Tibor Machan|chapter=Liberty, Equality, Solidarity Toward a Dialectical Anarchism|title=Anarchism/Minarchism: Is a Government Part of a Free Country?|pages=155–188|year=2008|isbn=978-0-7546-6066-8|publisher=Ashgate}}
* {{cite book|title=The Anarchists|last=Joll|first=James|author-link=James Joll|year=1964|publisher=Harvard University Press|isbn=978-0-674-03642-0}}
* {{cite journal|last1=Jun|first1=Nathan|title=Anarchist Philosophy and Working Class Struggle: A Brief History and Commentary|journal=[[WorkingUSA]]|volume=12|issue=3|pages=505–519|date=September 2009|language=en|doi=10.1111/j.1743-4580.2009.00251.x|issn=1089-7011}}<!-- to revisit, p. 508+ -->
* {{cite book|last=Jun|first=Nathan|chapter=The State|pages=27–47|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite journal|last=Kahn|first=Joseph|title=Anarchism, the Creed That Won't Stay Dead; The Spread of World Capitalism Resurrects a Long-Dormant Movement|year=2000|journal=[[The New York Times]]|issue=5 August}}
* {{cite book|last=Kinna|first=Ruth|author-link=Ruth Kinna|title=The Bloomsbury Companion to Anarchism|year=2012|publisher=Bloomsbury Academic|isbn=978-1-62892-430-5}}
* {{cite book|last=Kinna|first=Ruth|author-link=Ruth Kinna|title=The Government of No One, The Theory and Practice of Anarchism|publisher=[[Penguin Random House]]|url=https://books.google.com/books?id=xzeGDwAAQBAJ|year=2019|isbn=978-0-241-39655-1}}
* {{cite journal|last=Klosko|first=George|title=More than Obligation – William A. Edmundson: Three Anarchical Fallacies: An Essay on Political Authority|journal=The Review of Politics|volume=61|issue=3|year=1999|issn=1748-6858|doi=10.1017/S0034670500028989|pages=536–538}}
* {{cite book|last=Klosko|first=George|title=Political Obligations|url=https://books.google.com/books?id=ToHmfIj8d_gC|year=2005|publisher=Oxford University Press|isbn=978-0-19-955104-0}}
* {{cite journal|last=Kristjánsson|first=Kristján|title=Three Anarchical Fallacies: An Essay on Political Authority by William A. Edmundson|journal=Mind|volume=109|issue=436|pages=896–900|year=2000|jstor=2660038}}
* {{cite book|last=Laursen|first=Ole Birk|chapter=Anti-Imperialism|pages=149–168|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|chapter-url=https://books.google.com/books?id=SRyQswEACAAJ|year=2019|publisher=Springer|isbn=978-3-319-75620-2}}
* {{cite journal|last=Levy|first=Carl|s2cid=144317650|author-link=Carl Levy (political scientist)|title=Social Histories of Anarchism|journal=Journal for the Study of Radicalism|volume=4|issue=2|date=8 May 2011|issn=1930-1197|doi=10.1353/jsr.2010.0003|pages=1–44}}
* {{cite book|last=Long|first=Roderick T.|author-link=Roderick T. Long|editor-last1=Gaud|editor-first1=Gerald F.|editor-last2=D'Agostino|editor-first2=Fred|title=The Routledge Companion to Social and Political Philosophy|url=https://books.google.com/books?id=z7MzEHaJgKAC|year=2013|publisher=Routledge|isbn=978-0-415-87456-4}}
* {{cite book|last=Lucy|first=Nicholas|year=2020|chapter-url=https://books.google.com/books?id=-t7KDwAAQBAJ|chapter=Anarchism and Sexuality|title=The SAGE Handbook of Global Sexualities|publisher=SAGE Publishing|pages=160–183|isbn=9781529721942}}
* {{cite journal|last1=Lutz|first1=James M.|last2=Ulmschneider|first2=Georgia Wralstad|year=2019|title=Civil Liberties, National Security and U.S. Courts in Times of Terrorism|journal=Perspectives on Terrorism|volume=13|issue=6|pages=43–57|jstor=26853740}}
* {{cite book|last=Marshall|first=Peter|author-link=Peter Marshall (author)|title=Demanding the Impossible: A History of Anarchism|year=1992|isbn=978-0-00-217855-6|publisher=Harper Collins|location=London}}
* {{cite book|last=Marshall|first=Peter|author-link=Peter Marshall (author)|title=Demanding the Impossible: A History of Anarchism|year=1993|publisher=PM Press|place=Oakland, California|isbn=978-1-60486-064-1}}
* {{cite book|last=Mattern|first=Mark|chapter=Anarchism and Art|pages=589–602|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Mayne|first=Alan James|url=https://books.google.com/books?id=6MkTz6Rq7wUC&q=Communist+anarchism+believes+in+collective+ownership&pg=PA131|title=From Politics Past to Politics Future: An Integrated Analysis of Current and Emergent Paradigms|year=1999|publisher=Greenwood Publishing Group|accessdate=20 September 2010|isbn=978-0-275-96151-0}}
* {{cite book|last=McLaughlin|first=Paul|title=Anarchism and Authority: A Philosophical Introduction to Classical Anarchism|url=https://we.riseup.net/assets/394498/paul-mclaughlin-anarchism-and-authority-a-philosophical-introduction-to-classical-anarchism-1.pdf|archive-url=https://web.archive.org/web/20180804180522/https://we.riseup.net/assets/394498/paul-mclaughlin-anarchism-and-authority-a-philosophical-introduction-to-classical-anarchism-1.pdf|archive-date=4 August 2018|publisher=[[Ashgate Publishing|Ashgate]]|location=Aldershot|date=28 November 2007|isbn=978-0-7546-6196-2}}
* {{cite book|last1=Morland|first1=Dave|chapter=Anti-capitalism and poststructuralist anarchism|editor1=Jonathan Purkis|editor2=James Bowen|pages=23–38|title=Changing Anarchism: Anarchist Theory and Practice in a Global Age|url=https://books.google.com/books?id=etb2UFzCBv4C|year=2004|publisher=Manchester University Press|isbn=978-0-7190-6694-8}}
* {{cite book|last=Meltzer|first=Albert|author-link=Albert Meltzer|title=Anarchism: Arguments For and Against|url=https://archive.org/details/anarchism00albe|url-access=registration|date=1 January 2000|publisher=AK Press|isbn=978-1-873176-57-3}}
* {{cite book|last=Morris|first=Brian|title=Bakunin: The Philosophy of Freedom|url=https://books.google.com/books?id=GJXy5eCpPawC|date=January 1993|publisher=Black Rose Books|isbn=978-1-895431-66-7}}
* {{cite book|last=Morriss|first=Brian|year=2015|title=Anthropology, Ecology, and Anarchism: A Brian Morris Reader|others=Marshall, Peter|edition=illustrated|location=Oakland|publisher=PM Press|isbn=9781604860931}}
* {{cite book|last=Morris|first=Christopher W.|title=An Essay on the Modern State|url=https://books.google.com/books?id=uuyJ9Bw8w7QC|year=2002|publisher=Cambridge University Press|isbn=978-0-521-52407-0}}
* {{cite journal|last=Moynihan|first=Colin|title=Book Fair Unites Anarchists. In Spirit, Anyway|year=2007|journal=The New York Times|issue=16 April}}
* {{cite book|last=Moya|first=Jose C|editor=Geoffroy de Laforcade|others=Kirwin R. Shaffer|title=In Defiance of Boundaries: Anarchism in Latin American History|chapter-url=https://books.google.com/books?id=ikt6AQAACAAJ|year=2015|publisher=[[University Press of Florida]]|isbn=978-0-8130-5138-3|chapter=Transference, culture, and critique The Circulation of Anarchist Ideas and Practices}}
* {{cite journal|last=Nesser|first=Petter|year=2012|title=Research Note: Single Actor Terrorism: Scope, Characteristics and Explanations|journal=Perspectives on Terrorism|volume=6|issue=6|pages=61–73|jstor=26296894}}
* {{cite book|last=Nettlau|first=Max|author-link=Max Nettlau|title=A Short History of Anarchism|year=1996|publisher=Freedom Press|isbn=978-0-900384-89-9}}
* {{cite book|last=Newman|first=Michael|year=2005|title=Socialism: A Very Short Introduction|location=Oxford|publisher=Oxford University Press|isbn=9780192804310}}
* {{cite book|last=Newman|first=Saul|title=The Politics of Postanarchism|url=https://books.google.com/books?id=SiqBiViUsOkC&pg=PA43|year=2010|publisher=Edinburgh University Press|isbn=978-0-7486-3495-8}}
* {{cite book|last=Nicholas|first=Lucy|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=Springer|isbn=978-3-319-75620-2|chapter=Gender and Sexuality}}
* {{cite journal|last=Norris|first=Jesse J.|year=2020|title=Idiosyncratic Terrorism: Disaggregating an Undertheorized Concept|journal=Perspectives on Terrorism|volume=14|issue=3|jstor=26918296s|issn=2334-3745}}
* {{cite book|last=Nomad|first=Max|contribution=The Anarchist Tradition|editor1-last=Drachkovitch|editor1-first=Milorad M.|title=Revolutionary Internationals 1864–1943|publisher=Stanford University Press|page=88|year=1966|isbn=978-0-8047-0293-5}}
* {{cite journal|last=Osgood|first=Herbert L.|title=Scientific Anarchism|publisher=The Academy of Political Science|journal=Political Science Quarterly|volume=4|issue=1|date=March 1889|pages=1–36|jstor=2139424|doi=10.2307/2139424}}
* {{cite book|last=Parry|first=Richard|title=The Bonnot Gang|url=https://archive.org/details/bonnotgang0000parr|url-access=registration|year=1987|publisher=Rebel Press|isbn=978-0-946061-04-4}}
* {{cite book|last=Perlin|first=Terry M.|year=1979|title=Contemporary Anarchism|url=https://books.google.com/books?id=mppLKlwHx7oC|publisher=Transaction Publishers|isbn=978-1-4128-2033-2}}
* {{cite book|last=Pernicone|first=Nunzio|title=Italian Anarchism, 1864–1892|url=https://books.google.com/books?id=3ttgjwEACAAJ|year=2009|publisher=Princeton University Press|isbn=978-0-691-63268-1}}
* {{cite journal|last=Peterson|first=Steven A|title=Moral Development and Critiques of Anarchism|url=https://mises.org/library/moral-development-and-critiques-anarchism|journal=Journal of Libertarian Studies|volume=8|issue=2|pages=237–245|date=1987}}
* {{cite book|last=Pierson|first=Christopher|title=Just Property: Enlightenment, Revolution, and History|url=https://books.google.com/books?id=7jvKDAAAQBAJ&pg=PA187|year=2013|publisher=Oxford University Press|isbn=978-0-19-967329-2}}
* {{cite book|last=Ramnath|first=Maia|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|chapter-url=https://books.google.com/books?id=SRyQswEACAAJ|year=2019|publisher=Springer|isbn=978-3-319-75620-2|chapter=Non-Western Anarchisms and Postcolonialism}}
* {{cite book|last=Robé|first=Chris|title=Breaking the Spell: A History of Anarchist Filmmakers, Videotape Guerrillas, and Digital Ninjas|url=https://www.researchgate.net/publication/336710855|year=2017|publisher=PM Press|isbn=978-1-629-63233-9}}
* {{cite book|last=Rogers|first=Tristan J.|year=2020|title=The Authority of Virtue: Institutions and Character in the Good Society|location=London|publisher=Routledge|isbn=9781000222647}}
* {{cite book|last=Rupert|first=Mark|title=Globalization and International Political Economy|publisher=Rowman & Littlefield Publishers|location=Lanham|year=2006|isbn=978-0-7425-2943-4|url=https://archive.org/details/globalizationint00rupe}}
* {{cite book|last=Ryley|first=Peter|chapter=Individualism|pages=225–236|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Shannon|first=Deric|chapter=Anti-Capitalism and Libertarian Political Economy|pages=91–106|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Skirda|first=Alexandre|title=Facing the Enemy: A History of Anarchist Organization From Proudhon to May 1968|publisher=AK Press|year=2002|isbn=978-1-902593-19-7|title-link=Facing the Enemy}}
* {{cite book|last=Sylvan|year=2007|first=Richard|section=Anarchism|editor=Robert E. Goodin|editor2=Philip Pettit|editor3=Thomas Pogge|title=A Companion to Contemporary Political Philosophy|edition=2nd|url=http://eltalondeaquiles.pucp.edu.pe/wp-content/uploads/2016/04/Robert-E--Goodin-Philip-Pettit-Thomas-W--Pogge-A-Companion-to-Contemporary-Political-Philosophy-2-Volume-Set-Blackwell-Companions-to-Philosophy-2007.pdf|archive-url=https://web.archive.org/web/20170517032711/http://eltalondeaquiles.pucp.edu.pe/wp-content/uploads/2016/04/Robert-E--Goodin-Philip-Pettit-Thomas-W--Pogge-A-Companion-to-Contemporary-Political-Philosophy-2-Volume-Set-Blackwell-Companions-to-Philosophy-2007.pdf|archive-date=17 May 2017|series=Blackwell Companions to Philosophy|volume=5|publisher=Blackwell Publishing|isbn=978-1-4051-3653-2|author-link=Richard Sylvan|editor1-link=Robert E. Goodin|editor2-link=Philip Pettit|editor3-link=Thomas Pogge}}
* {{cite book|last=Suissa|first=Judith|chapter=Anarchist Education|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|chapter-url=https://books.google.com/books?id=SRyQswEACAAJ|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Thomas|first=Paul|title=Karl Marx and the Anarchists|publisher=Routledge & Kegan Paul|location=London|year=1985|isbn=978-0-7102-0685-5}}
* {{cite book|last=Turcato|first=Davide|chapter=Anarchist Communism|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite book|last=Van der Walt|first=Lucien|chapter=Syndicalism|pages=249–264|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=[[Springer Publishing]]|isbn=978-3-319-75620-2}}
* {{cite journal|last=Ward|first=Colin|date=1973|title=The Role of the State|journal=Education Without Schools|pages=39–48}}
* {{cite book|last=Ward|first=Colin|author-link=Colin Ward|year=2004|url=https://books.google.com/books?id=nkgSDAAAQBAJ|title=Anarchism: A Very Short Introduction|location=Oxford|publisher=Oxford University Press|isbn=978-0-19-280477-8}}
* {{cite book|last=Walter|first=Nicholas|year=2002|title=About Anarchism|location=London|publisher=Freedom Press|isbn=9780900384905}}
* {{cite journal|last=Wendt|first=Fabian|year=2020|title=Against Philosophical Anarchism|journal=Law and Philosophy|volume=39|issue=5|pages=527–544|doi=10.1007/s10982-020-09377-4}}
* {{cite book|last=Wilbur|first=Shawn|chapter=Mutualism|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=Springer Publishing|isbn=978-3-319-75620-2}}
* {{cite journal|last=Williams|first=Dana M.|year=2015|title=Black Panther Radical Factionalization and the Development of Black Anarchism|journal=Journal of Black Studies|location=Thousand Oaks|publisher=SAGE Publishing|volume=46|issue=7|pages=678–703|doi=10.1177/0021934715593053|jstor=24572914|s2cid=145663405}}
* {{cite journal|last=Williams|first=Dana M.|title=Contemporary Anarchist and Anarchistic Movements|journal=Sociology Compass|publisher=Wiley|volume=12|issue=6|pages=e12582|year=2018|issn=1751-9020|doi=10.1111/soc4.12582}}
* {{cite book|last=Williams|first=Dana M.|chapter=Tactics: Conceptions of Social Change, Revolution, and Anarchist Organisation|editor1-last=Levy|editor1-first=Carl|editor1-link=Carl Levy (political scientist)|editor2-last=Adams|editor2-first=Matthew S.|title=The Palgrave Handbook of Anarchism|year=2019|publisher=Springer Publishing|isbn=978-3-319-75620-2}}
* {{cite journal|last=Williams|first=Leonard|year=2010|title=Hakim Bey and Ontological Anarchism|journal=Journal for the Study of Radicalism|location=East Lansing|publisher=Michigan State University Press|volume=4|issue=2|pages=109–137|doi=10.1353/jsr.2010.0009|jstor=41887660|s2cid=143304524}}
{{refend}}

==== Tertiary sources ====
{{refbegin|35em|indent=yes}}
* {{cite web|url=http://www.allmusic.com/style/anarchist-punk-ma0000011967|title=Anarchist Punk|website=AllMusic|accessdate=1 August 2020|ref={{harvid|AllMusic}}}}
* {{cite web|last=Coutinho|first=Steve|title=Zhuangzi|publisher=Internet Encyclopedia of Philosophy|date=3 March 2016|url=http://www.iep.utm.edu/zhuangzi/|archive-url=https://web.archive.org/web/20160303175106/http://www.iep.utm.edu/zhuangzi/|archive-date=3 March 2016|url-status=live|accessdate=5 March 2019}}
* {{cite book|last=De George|first=Richard T.|editor=Ted Honderich|editor-link=Ted Honderich|title=The Oxford Companion to Philosophy|publisher=[[Oxford University Press]]|isbn=9780199264797|year=2005}}
* {{cite encyclopedia|last=Fiala|first=Andrew|title=Anarchism|encyclopedia=[[Stanford Encyclopedia of Philosophy]]|year=2017|url=https://plato.stanford.edu/entries/anarchism/|publisher=Metaphysics Research Lab, Stanford University}}
* {{cite encyclopedia|last1=Miller|first1=Martin A.|last2=Dirlik|first2=Arif|last3=Rosemont|first3=Franklin|last4=Augustyn|first4=Adam|last5=Duignan|first5=Brian|last6=Lotha|first6=Gloria|title=Anarchism In The Arts|encyclopedia=[[Encyclopaedia Britannica]]|year=2019|url=https://www.britannica.com/topic/anarchism/Anarchism-in-the-arts#info-article-contributors}}
* {{cite encyclopedia|last1=Morriss|first1=Andrew|editor-last1=Hamowy|editor-first1=Ronald|editor-link=Ronald Hamowy|title=Anarcho-capitalism|pages=13–14|year=2008|url=https://books.google.com/books?id=yxNgXs3TkJYC&pg=PT51|isbn=978-1-4129-6580-4|publisher=SAGE; Cato Institute|encyclopedia=The Encyclopedia of Libertarianism|oclc=191924853|doi=10.4135/9781412965811.n8}}
* {{cite book|last1=McLean|first1=Iain|first2=Alistair|last2=McMillan|title=The Concise Oxford Dictionary of Politics|url=https://archive.org/details/oxfordconcisedic00iain|url-access=registration|year=2003|publisher=Oxford University Press|isbn=978-0-19-280276-7}}
* {{cite web|title=Definition of Anarchism|work=Merriam-Webster|year=2019|url=https://www.merriam-webster.com/dictionary/anarchism|ref={{sfnref|Merriam-Webster|2019}}|access-date=28 February 2019}}
* {{cite book|last=Miller|first=David|title=The Blackwell Encyclopaedia of Political Thought|url=https://books.google.com/books?id=NIZfQTd3nSMC|date=26 August 1991|publisher=Wiley|isbn=978-0-631-17944-3}}
* {{cite encyclopedia|last=Ostergaard|first=Geoffrey|author-link=Geoffrey Ostergaard|year=2003|url=https://books.google.com/books?id=JJmdpqJwkwwC&pg=PA12|title=Anarchism|editor-last=Outhwaite|editor-first=William|encyclopedia=The Blackwell Dictionary of Modern Social Thought|edition=2nd|location=Malden, Massachusetts|publisher=Blackwell Publishing|isbn=978-0-631-22164-7|oclc=49704935}}
* {{cite book|chapter=Anarchy|title=Oxford English Dictionary|edition=3rd|publisher=Oxford University Press|date=September 2005|ref={{harvid|Oxford English Dictionary|2005}}}}
{{refend}}

== Further reading ==
* {{cite book|last=Barclay|first=Harold B.|author-link=Harold Barclay|title=People Without Government: An Anthropology of Anarchy|url=https://books.google.com/books?id=MrFHQgAACAAJ|year=1990|publisher=Kahn & Averill|isbn=978-0-939306-09-1}}
* {{cite book|last=Edmundson|first=William A.|title=Three Anarchical Fallacies: An Essay on Political Authority|url=https://books.google.com/books?id=q_gClKUbJyYC|date=2007|publisher=Cambridge University Press|isbn=978-0-521-03751-8}} Criticism of philosophical anarchism.
* {{cite book|last=Harper|first=Clifford|author-link=Clifford Harper|year=1987|url=https://books.google.com/books?id=W63aAAAAMAAJ|title=Anarchy: A Graphic Guide|publisher=Camden Press|isbn=978-0-948491-22-1}}
* {{cite book|last=Huemer|first=Micheal|year=2012|title=The Problem of Political Authority: An Examination of the Right to Coerce and the Duty to Obey|location=London|publisher=Palgrave Macmillan|isbn=9781137281647}} A defence of philosophical anarchism, arguing that "both kinds of 'anarchism' [i.e. philosophical and political anarchism] are philosophical and political claims." (p. 137)
* {{cite book|last=Le Guin|first=Ursula K.|author-link=Ursula K. Le Guin|title=The Dispossessed|year=2009|publisher=HarperCollins|title-link=The Dispossessed}} Anarchistic popular fiction novel.<!-- Gifford 2019, p 580. -->
* {{cite book|last=Kinna|first=Ruth|author-link=Ruth Kinna|title=Anarchism: A Beginners Guide|url=https://books.google.com/books?id=LLLaAAAAMAAJ|year=2005|publisher=Oneworld|isbn=978-1-85168-370-3}}
* {{cite book|last=Sartwell|first=Crispin|author-link=Crispin Sartwell|title=Against the State: An Introduction to Anarchist Political Theory|publisher=SUNY Press|year=2008|isbn=978-0-7914-7447-1|url=https://books.google.com/books?id=bk-aaMVGKO0C}}
* {{cite book|last=Scott|first=James C.|author-link=James C. Scott|year=2012|title=Two Cheers for Anarchism: Six Easy Pieces on Autonomy, Dignity, and Meaningful Work and Play|title-link=Two Cheers for Anarchism|location=Princeton, New Jersey|publisher=Princeton University Press|isbn=978-0-691-15529-6}}
* {{cite book|last=Wolff|first=Robert Paul|author-link=Robert Paul Wolff|year=1998|title=In Defense of Anarchism|title-link=In Defense of Anarchism|publisher=University of California Press|isbn=978-0-520-21573-3}} An argument for philosophical anarchism.
* {{cite journal|last=Woodcock|first=George|author-link=George Woodcock|date=January 1962|url=https://www.historytoday.com/archive/anarchism-spain|title=Anarchism in Spain|journal=History Today|volume=12|issue=1|pages=22–32|accessdate=14 October 2020}}

== External links ==
{{sister project links|voy=no|n=no|v=no|b=Subject:Anarchism|s=Portal:Anarchism|d=Q6199|c=Category:Anarchism}}
* [http://dwardmac.pitzer.edu/ Anarchy Archives]. [[Anarchy Archives]] is an online research center on the history and theory of anarchism
<!-- Attention! The external link portion of this article regularly grows far beyond manageable size. PLEASE only list an outside link if it applies to anarchism in general, is somewhat noteworthy, and has consensus on the talkpage. Links to sites which cover anarchist submovements will be routinely moved to subarticles to keep this article free of clutter. -->
{{prone to spam|date=November 2014}}
{{Z148}}<!-- {{no more links}}. Please be cautious adding more external links. Wikipedia is not a collection of links and should not be used for advertising. Excessive or inappropriate links will be removed. See [[Wikipedia:External links]] and [[Wikipedia:Spam]] for details. If there are already suitable links, propose additions or replacements on the article's talk page, or submit your link to the relevant category at Curlie (curlie.org) – and link there using {{curlie}}. -->

{{-}}
{{anarchism}}
{{anarchies}}
{{libertarian socialism}}
{{libertarianism}}
{{philosophy topics}}
{{political culture}}
{{political ideologies}}
{{social and political philosophy}}
{{authority control}}
[[Category:Anarchism| ]]
[[Category:Anti-capitalism]]
[[Category:Anti-fascism]]
[[Category:Economic ideologies]]
[[Category:Left-wing politics]]
[[Category:Libertarian socialism]]
[[Category:Libertarianism]]
[[Category:Political culture]]
[[Category:Political movements]]
[[Category:Political ideologies]]
[[Category:Social theories]]
[[Category:Socialism]]
"""

wikitext_two = """
{{Infobox person
| name        = Felix Rexhausen
| image       = Rexhausen.jpg
| caption = Rexhausen ca. 1984. 
| birth_date  = {{Birth date|df=y|1932|12|31}}
| birth_place = [[Cologne, Germany]]
| death_date  = {{Death date and age|df=y|1992|02|06|1932|12|31}}
| death_place = [[Hamburg, Germany]]
| occupation  = Magazine editor, journalist, author
| alias       = Stefan David
| credits     = ''[[Westdeutscher Rundfunk|WDR]]'', ''Cologne Stadtanzeiger''
}}

'''Felix Rexhausen''' (31 December 1932 – 6 February 1992) was a [[Germany|German]] journalist, editor and author. As a journalist, he wrote for ''Kölner Stadt-Anzeiger'', ''[[Westdeutscher Rundfunk]]'', and the magazines ''[[Die Zeit]]'' and ''[[Der Spiegel]]''.

==Biography==
Rexhausen lived in [[Leipzig]] and [[Hamburg]] during his childhood. He studied [[Economics]] at the [[University of Cologne]], where he finished 1959. He then worked as an editor for the ''Westdeutscher Rundfunk'' in Cologne, and later was a journalist for ''Kölner Stadt-Anzeiger'', ''Der Spiegel'' and ''Die Zeit''. Together with [[Carola Stern]] and [[Gerd Ruge]], Rexhausen founded the German chapter of [[Amnesty International]] in 1961. After 1968, Rexhausen worked as an author in Hamburg, writing novels, satires and lyric poetry. One of his themes is [[homosexuality]]. Rexhausen died on 6 February 1992 in Hamburg.{{citation needed|date=August 2015}}

The German National Lesbian and Gay Journalists Association<ref>[http://www.blsj.de/ BLSJ]</ref> has given an award for journalists who write on [[LGBT]] topics since 1998. The name of the award is the [[Felix-Rexhausen-Award]]<ref>[http://www.felix-rexhausen-preis.de/ Felix-Rexhausen-Award]</ref> after Rexhausen, who was [[gay]].<ref name="hergemoller">{{citation|title=Einführung in die Historiographie der Homosexualitäten|first=Bernd-Ulrich|last=Hergemöller|year=1999|publisher=dition Diskord|isbn=3-89295-678-2}}</ref>

==Works==
*''Der Unternehmer und die volkswirtschaftliche Entwicklung'', Berlin 1960.
*''Die Finanzpublizität der Länder und Gemeinden'', Berlin 1963.
*''Mit Bayern leben'', Offenbach am Main 1963.
*''Der linkische und der Weg zum Rechts-Staat oder Wer zersetzt hier was?'', Cologne 1965.
*''Mit deutscher Tinte'', Frankfurt a.M. 1965.
*''Lavendelschwert'', Frankfurt a.M. 1966.
*''Gedichte an Bülbül'', Stierstadt i.Ts. 1968.
*''Die Sache'', Frankfurt/M. 1968.
*''Berührungen'', Darmstadt, 1969 (under alias Stefan David).
*''Seelenadel'', Zweibrücken, 1969 (with [[Axel Hertenstein]]).
*''Von großen Deutschen'', Stierstadt i.Ts., 1969
*''Germania unter der Gürtellinie'', Bern,  1970
*''Spukspaßspitzen'', Pforzheim 1970, (with Axel Hertenstein)
*''Wie es so geht'', Düsseldorf, 1974
*''So und so'', Leverkusen, 1976
*'In Harvestehude'', Hamburg, 1979
*''Die Lavendeltreppe'', Düsseldorf, 1979
*''Über Wahlkampf'', Luzern, 1980
*''Beste Fahrt!'', Ottersberg, 1981
*''Gesammelte Werke'', Berlin
**Band 1: Die Märchenklappe: allerlei Zwischenmännlichkeiten; Geschichten, Mären, Reime, 1982
*''Der heutige Homosexuelle und Weihnachten'', Kiel, 1987
*''Die deutsche Weihnacht'', Munich, 1989
*''Wenn der Schafvogel schreit'', Klagenfurt, 1999 (under alias Stefan David)

==Notes==
{{Reflist}}

{{Authority control}}

{{DEFAULTSORT:Rexhausen, Felix}}
[[Category:1932 births]]
[[Category:1992 deaths]]
[[Category:Journalists from Hamburg]]
[[Category:German male journalists]]
[[Category:20th-century German journalists]]
[[Category:Gay writers]]
[[Category:LGBT journalists from Germany]]
[[Category:Writers from Cologne]]
[[Category:Westdeutscher Rundfunk people]]
[[Category:Die Zeit people]]
"""

In [7]:
def removeComments(wikitext):
    return re.sub('<!--.*?-->', '', wikitext, flags=re.DOTALL)

spark.udf.register('removeComments', removeComments, 'String')

In [ ]:
# Example test of verifying same output
# Comments are a major source of differences so these are filtered out
wt = mwparserfromhell.parse(wikitext)
mwp = getTemplatesMWP(removeComments(wikitext))
reg = getTemplatesRegex(removeComments(wikitext))
for l in mwp:
    if l in reg:
        reg.pop(reg.index(l))
    else:
        print('{0} in MWP only.'.format(l))
for l in reg:
    print('{0} in Regex only.'.format(l))

In [30]:
wt = mwparserfromhell.parse(wikitext_two)